# 🎇 LangChain

\[[Chinese Version](./LangChainTutorial.ipynb)\]

How to create our own exclusive chatbot? How to learn domain-specific knowledge through simple manipulation of large language models (LLMs)? How to unleash more potential in daily workflow using LLMs?
>Please imagine the following scenarios: you have several e-books or dozens of text files, or you complete specific tasks using a database. We want the LLMs model to learn the data provided by the user and only answer relevant questions within the given data range. If the question exceeds the scope, inform the user that the question is beyond the scope and cannot be answered. In other words, we want to restrict the LLMs model from freely expressing itself and prevent it from speaking casually. How can we accomplish the above tasks based on the large model? LangChain can help you achieve this. Click 👉[here](https://python.langchain.com/en/latest/index.html)👈 to directly access the official documentation of LangChain.

LangChain is an upper-level tool for large models, an application development framework based on LLMs that utilizes composability to build applications with LLMs. The emphasis is on "composability". It designs a series of interfaces that are easy to integrate into real-world applications, reducing the difficulty of deploying large language models in practical scenarios. LangChain can be used for chatbots, generative question answering (GQA), text summarization, and more.
The goal of LangChain is:
- Allowing large language models to handle data from different sources.
- Enabling interaction between large language models and the environment in which they are deployed.

<img src="./langchain.png" align=center width=100% />

As shown in the figure above, the LangChain library consists of six main components:
- **Models**: Provides pre-packaged large models based on the OpenAI API, including common OpenAI models, and also supports custom encapsulation of large models.
- **Prompt**: Supports rapid implementation of custom prompt projects and integration with LLMs.
- **Index**: Accepts user queries and returns the most relevant content.
- **Memory**: A standard interface for storing state between chains/calls.
- **Chains**: A series of calls (LLMs or others, such as networks, operating systems). Chains provide standard interfaces and settings to combine these calls. Information is first obtained from external sources and then fed to LLMs. The large model sequentially executes logical chains for a series of tasks.
- **Agents**: Agents play a crucial role in determining the actions to be taken with LLMs and how to perform them. Typically, capabilities in Utils and various logical chains in Chains are encapsulated as tools for intelligent invocation by Agents.

🌟 We will primarily focus on code explanations and demonstrations using the OpenAI provider. So let's begin this journey together! ✊

# 📜 CATLOG

- [Before Start](#before-start)
- [Models](#models)
- [Prompt](#prompt)
- [Index](#index)
- [Memory](#memory)
- [Chains](#chains)
- [Agents](#agents)
- [Coding Examples](#coding-exampls)

## Before Start

Environment configuration and proxy settings are for domestic users (specifically for mainland China, not including Hong Kong, Macau, and Taiwan regions) who need to use VPN to access OpenAI. Therefore, it is necessary to run the program locally to set up the proxy and avoid any network-related access and invocation issues.

In [67]:
import os
# Set up the HTTP proxy, local IP address, and port number. 
# You can view the IP address using the IPCONFIG/IFCONFIG command on Win/Linux. 
# Alternatively, you can use the default setting of 127.0.0.1 for the IP address 
# and specify the port number of the mounted VPN.
os.environ['HTTP_PROXY'] = 'http://127.0.0.1:XXX'
# Set up the HTTPS proxy, same as above.
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:XXX'

#check the proxy is useful or not
def check_proxy():
    import urllib.request
    url = "https://www.google.com"
    # url = "https://www.baidu.com"
    filename = "google.html"
    urllib.request.urlretrieve(url, filename)#保存在当前文件夹下

check_proxy()

# openai key
os.environ["OPENAI_API_KEY"] = "填上你自己的openai api  key"

#search api key, only using in agent, this step is optional
os.environ['SERPAPI_API_KEY']='填上你自己的serpapi api key'

Check if the Google Search API is available. This step is optional and not necessary. If you are using the Google Search API in the following examples, you will need to register and obtain the necessary credentials to make the API calls. You can register by clicking [here](https://serpapi.com/dashboard).

In [8]:
from serpapi import GoogleSearch
search = GoogleSearch({
    "q": "coffee",
    "location": "Austin,Texas",
  })
result = search.get_dict()
print(result)

{'search_metadata': {'id': '643802169d158690e4963190', 'status': 'Success', 'json_endpoint': 'https://serpapi.com/searches/11aa25012aaee788/643802169d158690e4963190.json', 'created_at': '2023-04-13 13:22:30 UTC', 'processed_at': '2023-04-13 13:22:30 UTC', 'google_url': 'https://www.google.com/search?q=coffee&oq=coffee&uule=w+CAIQICIdQXVzdGluLFRYLFRleGFzLFVuaXRlZCBTdGF0ZXM&sourceid=chrome&ie=UTF-8', 'raw_html_file': 'https://serpapi.com/searches/11aa25012aaee788/643802169d158690e4963190.html', 'total_time_taken': 6.54}, 'search_parameters': {'engine': 'google', 'q': 'coffee', 'location_requested': 'Austin,Texas', 'location_used': 'Austin,TX,Texas,United States', 'google_domain': 'google.com', 'device': 'desktop'}, 'search_information': {'organic_results_state': 'Results for exact spelling', 'query_displayed': 'coffee', 'total_results': 4660000000, 'time_taken_displayed': 0.68, 'menu_items': [{'position': 1, 'title': 'Images', 'link': 'https://www.google.com/search?q=coffee&source=lnms&t

check the OpenAI KEY is useful or not

In [2]:
import openai

response = openai.Completion.create(
    engine="davinci",
    prompt="Hello, my name is",
    max_tokens=5,
    n=1,
    stop=None,
    temperature=0.5,
)
print("response : {}".format(response))
print("--------")
print(response.choices[0].text)

response : {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": " Kaitlin. I"
    }
  ],
  "created": 1682500219,
  "id": "cmpl-79VSFbbtZ9ekxWgW9LTDSvimSJwd3",
  "model": "davinci",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 5,
    "prompt_tokens": 5,
    "total_tokens": 10
  }
}
--------
 Kaitlin. I


---

## Models

So now we begin our first section of content, the code examples in this section are sourced from the official LangChain tutorial website. For detailed instructions and documentation on Models, please click 👉[Here](https://python.langchain.com/en/latest/modules/models.html)👈 for direct access.

- [LLMs](#llms)
- [Chat Models](#chat-models)
- [Text Embedding Models](#text-embedding-models)

### LLMs

We know that LangChain itself does not provide large language models, but rather provides various interfaces and methods for accessing and calling upon large language models. Currently, there are many LLM providers such as OpenAI, Cohere, Hugging Face, etc. Within these LLMs, a unified interface is provided to access these large models.

In [5]:
from langchain.llms import OpenAI
import os


Here are the regular ways to call and declare LLMs, there are many models to choose from, you can visit [here](https://platform.openai.com/docs/models/overview) for browsing and setting which specific model to use.

In [6]:
"""model_name is the name of the selected OpenAI model, 
n=2 means that two pieces of text are generated and best_of selects the best one out of the two as the output.
"""
llm = OpenAI(model_name="text-ada-001", n=2, best_of=2)

So let's take a look at the specific parameters of this LLM.

In [7]:
llm

OpenAI
Params: {'model_name': 'text-ada-001', 'temperature': 0.7, 'max_tokens': 256, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 2, 'best_of': 2, 'request_timeout': None, 'logit_bias': {}}


Text Generation: The most basic function of an LLM is to be able to call it, pass in a string and return a string. The "tell me...story" is the query, which is passed into the model and called upon to answer the question. The query type is a string and there is no restriction on whether it must be a question or statement. Large language models such as ChatGPT will parse and understand the content accordingly, so readers need not worry about this aspect.

In [6]:
llm("Tell me a scared story")

"\n\nOnce upon a time there was a princess who was very scared. She was sitting in her bedroom when she saw a scary scene in a movie. The princess was scared and started to cry. Her parents tried to soothe her and eventually she fell asleep. When she woke up, she found herself in a dark place. She couldn't make head or tail of what was going on and started to scream. A big part of her population felt sorry for her and helped her, but she wasn't alone. The people who were supposed to be helping her started to leave without her. The princess was scared and started to run around. She didn't know where she was going or what she was doing. Eventually she got her hands on a knife and started to kill people. She was scared and full of Syndicate, but she didn't care. The princess died in the dark, but her story is still shareable online."

LLM can accept a list of texts and generate answers for each statement, including the return information from API providers.

In [8]:
llm_result = llm.generate(["How to study English", "How to generate a new idea"]*3)

In [9]:
llm_result

generations=[[Generation(text='\n\nThere are a few ways to study English:\n\n1. Use online resources.\n\n2. Take classes from an English teacher.\n\n3. Use English-speaking books and articles.\n\n4. Take English-speaking courses at your school.', generation_info={'finish_reason': 'stop', 'logprobs': None}), Generation(text=' in a foreign country\n\nThere are a number of ways to study English in a foreign country. One way is to go to a training course offered by a English speaking teacher. Another way is to use online resources.', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text="\n\n1. Take a step back. What is the best idea you've heard?\n2. Ask yourself how it would be used. Is the idea unique to you, or is it something that is commonly used?\n3. How would you create it? This might be a difficult question to answer. Try to think of ways to create the idea that is most unique to you.\n4. What ingredients would you need? The ingredients for the idea might

In [12]:
len(llm_result.generations)

6

In [17]:
llm_result.generations[0]

[Generation(text='\n\n1. Open a English course in your local university or department.\n\n2. Find a course on English you want to take and take it in your local university.\n\n3. Find a tutor or course Tutor English for you to learn from.\n\n4. Finish your coursework and you will have completed English studies in your local university.', generation_info={'finish_reason': 'stop', 'logprobs': None}),
 Generation(text='\n\nThere are a few ways to study English:\n\n1. Online courses: Take online courses from various English schools.\n\n2. Practice tests: Performance test materials and practice questions in real time so you can become comfortable with the material.\n\n3. Print books: Read print books and learn key concepts in the same way as if they were video lessons.\n\n4. Location: Take classes and workshops near you at a nearby school or organization.\n\n5. Online courses: Take online courses that interest you. Check out free courses beforeempying to sink in for real.', generation_info=

In [18]:
llm_result.generations[1]

[Generation(text='\n\n\nThere are a few ways to generate new ideas:\n\n1. Look at what others have done in their line of work and experiment with different ideas.\n\n2. Take a step back and look at the whole work process from a different perspective.\n\n3. Think about different ways to do a task that is popular or efficient in another job market.\n\n4.Solution:\n\nTake a step back and think about different ways to do a task that is popular or efficient in another job market. For example, think about how to automate task completion times in a specific industry.', generation_info={'finish_reason': 'stop', 'logprobs': None}),
 Generation(text=' for a game\n\nThere is no one definitive way to generate a new idea for a game. However, some methods you may consider include searching online forums for various ideas, creating a list of specific ideas you have, or searching Google for " game idea" resources. Additionally, if you have any specific ideas you would like to see included in this reso

Provide the result from API providers.

In [102]:
llm_result.llm_output

{'token_usage': {'prompt_tokens': 42,
  'completion_tokens': 1407,
  'total_tokens': 1449},
 'model_name': 'text-ada-001'}

evaluation for a array by costing the number of tokens 

In [103]:
llm.get_num_tokens("假设你是个教师，如何教英语")

29

### Chat Models
The turbo model is used in the ChatGPT interface. There are three important roles in the design of a chatbot: system, human, and AI. The image below shows the capture and processing of information by the chatbot:

![Chatbot Information Capture and Processing](./three.png)

As depicted in the image, these three roles are defined as follows:
- System: Describes everything in the real world. The system determines what the AI ultimately needs to do and provides a clear context or role, such as a translator or banker.
- Human: Specifies specific tasks or requests, such as a sentence for the AI to translate or a request for the AI to generate lyrics resembling those of Fang Wenshan.
- AI: The large model that decides the content to be returned. This functionality is implemented using the langchain toolkit.

In the code, we manipulate specific requirements and steps using the SystemMessage, HumanMessage, and AIMessage imported from the langchain.schema, as shown below:

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate, LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [4]:
chat = ChatOpenAI(temperature=0)
chat

ChatOpenAI(verbose=False, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x7fad12836580>, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-3.5-turbo', temperature=0.0, model_kwargs={}, openai_api_key=None, openai_organization=None, request_timeout=60, max_retries=6, streaming=False, n=1, max_tokens=None)

the response is generated by AI

In [12]:
chat([HumanMessage(content="Translate this sentence from English to Chinese. I love programming.")])

AIMessage(content='我喜欢编程。', additional_kwargs={})

As shown in the image above, we typically define the content of SystemMessage as assigning it a role so that the OpenAI model can answer within a specific domain of knowledge. The benefit of doing this is that it makes the answers provided more accurate. It is worth noting that the OpenAI chat model supports the input of multiple messages at once.

In [8]:
messages = [
    SystemMessage(content="You are a helpful assistant that translates English to Chinese."),
    HumanMessage(content="Translate this sentence from English to Chinese. I love programming.")
]
chat(messages)

AIMessage(content='我喜欢编程。', additional_kwargs={})

you can use ```chat.generate``` to deal with the multiple dialogue, and LLMResult including the input info.

In [9]:
batch_messages = [
    [
        SystemMessage(content="You are a helpful assistant that translates English to French."),
        HumanMessage(content="Translate this sentence from English to French. I love programming.")
    ],
    [
        SystemMessage(content="You are a helpful assistant that translates English to Chinese."),
        HumanMessage(content="Translate this sentence from English to Chinese. I love artificial intelligence.")
    ],
]
result = chat.generate(batch_messages)
result

LLMResult(generations=[[ChatGeneration(text="J'aime programmer.", generation_info=None, message=AIMessage(content="J'aime programmer.", additional_kwargs={}))], [ChatGeneration(text='我喜欢人工智能。', generation_info=None, message=AIMessage(content='我喜欢人工智能。', additional_kwargs={}))]], llm_output={'token_usage': {'prompt_tokens': 73, 'completion_tokens': 16, 'total_tokens': 89}, 'model_name': 'gpt-3.5-turbo'})


OUT：

LLMResult(generations=[[ChatGeneration(text="J'aime programmer.", generation_info=None, message=AIMessage(content="J'aime programmer.", additional_kwargs={}))], [ChatGeneration(text='我喜欢人工智能。', generation_info=None, message=AIMessage(content='我喜欢人工智能。', additional_kwargs={}))]], llm_output={'token_usage': {'prompt_tokens': 73, 'completion_tokens': 16, 'total_tokens': 89}, 'model_name': 'gpt-3.5-turbo'})



we can obtain the info from result as below:

In [109]:
result.llm_output

{'token_usage': {'prompt_tokens': 73,
  'completion_tokens': 16,
  'total_tokens': 89},
 'model_name': 'gpt-3.5-turbo'}

### Use the prompt template in ChatModel


use the template to pack the SystemMessage and HumanMessage.


In [16]:
template="You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [17]:
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
chat(chat_prompt.format_prompt(input_language="English", output_language="French", text="I love programming.").to_messages())

AIMessage(content="J'adore la programmation.", additional_kwargs={})


you can also adopt the methods below to better construct ```system_message``` directly

In [12]:
prompt=PromptTemplate(
    template="You are a helpful assistant that translates {input_language} to {output_language}.",
    input_variables=["input_language", "output_language"],
)
prompt

PromptTemplate(input_variables=['input_language', 'output_language'], output_parser=None, partial_variables={}, template='You are a helpful assistant that translates {input_language} to {output_language}.', template_format='f-string', validate_template=True)

In [14]:
system_message_prompt = SystemMessagePromptTemplate(prompt=prompt)
system_message_prompt

SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input_language', 'output_language'], output_parser=None, partial_variables={}, template='You are a helpful assistant that translates {input_language} to {output_language}.', template_format='f-string', validate_template=True), additional_kwargs={})

### LLMChain

结合LLMChain和Prompt、Chat model，可以更方便地开展对话

In [19]:
chain = LLMChain(llm=chat, prompt=chat_prompt)
chain

LLMChain(memory=None, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x7fad12836580>, verbose=False, prompt=ChatPromptTemplate(input_variables=['text', 'output_language', 'input_language'], output_parser=None, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input_language', 'output_language'], output_parser=None, partial_variables={}, template='You are a helpful assistant that translates {input_language} to {output_language}.', template_format='f-string', validate_template=True), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], output_parser=None, partial_variables={}, template='{text}', template_format='f-string', validate_template=True), additional_kwargs={})]), llm=ChatOpenAI(verbose=False, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x7fad12836580>, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, mo

In [27]:
chain.run(input_language="English", output_language="Chinese", text="I love programming and swimming")

'我喜欢编程和游泳。'

### 流式对话
通过回调函数处理对话

In [30]:
from langchain.callbacks.base import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
"""
streaming=True是启用了聊天的流式模式，也就是响应的内容将会在逐步生成的数据块中逐步返回；
而callback_manger用于管理聊天过程中的回调函数的对象，流式对象将会逐步返回内容而不是整个
响应完成之后一次性返回，如果不传递callback_manager参数，则聊天过程中将会以默认的方式运行，也
就是响应完成之后一次性返回；verbose是详细模式，决定是否打印出具体的信息；temperature的设置
则是决定随机性
"""
chat = ChatOpenAI(streaming=True, callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]), verbose=True, temperature=0)
resp = chat([HumanMessage(content="帮我分析一下为什么一加一等于二？")])

一加一等于二是因为这是我们所接受的基本数学原理之一。在十进制数系统中，我们将数字1表示为一个单位，数字2表示为两个单位。当我们将一个单位与另一个单位相加时，我们得到两个单位，即数字2。这是因为加法是一种基本的算术运算，它表示将两个或多个数值相加以得到它们的总和。因此，一加一等于二是数学中的基本原理之一，它被广泛接受并被用于各种数学和科学应用中。

In [29]:
resp_another = chat([HumanMessage(content="tell me how to become a superman?")])

As an AI language model, I cannot provide instructions on how to become a fictional character like Superman. However, I can suggest some ways to become a better version of yourself:

1. Set goals and work towards them consistently.
2. Exercise regularly to improve physical strength and endurance.
3. Learn new skills and knowledge to enhance mental abilities.
4. Practice empathy and kindness towards others.
5. Be courageous and stand up for what is right.
6. Develop a positive mindset and attitude towards life.
7. Surround yourself with supportive and positive people.
8. Take care of your health by eating a balanced diet and getting enough rest.
9. Continuously challenge yourself to grow and improve.
10. Believe in yourself and your abilities.

### Text Embedding Models
词嵌入支持将自然语言转变为向量。LangChain中最重要的两个embedding方法为embed_documents 和 embed_query方法。因为处理多个文本文件和处理一个文件有很大的不同，所以LangChain将其分为两个类。

LangChain中集成了很多来自不同平台的API可以进行词嵌入编码，这里举一个用OpenAi进行词嵌入的方法

In [31]:
#我们使用较多的也是OpenAI的embeddings方法
from langchain.embeddings import OpenAIEmbeddings

In [32]:
embeddings = OpenAIEmbeddings()

In [33]:
text = "This is a test document."

In [43]:
query_result = embeddings.embed_query(text)
query_result

[-0.0031265460635144116,
 0.011133635493097378,
 -0.004037691773639618,
 -0.011746618046080886,
 -0.000993598608605281,
 0.01080715570571065,
 -0.010440697965211392,
 -0.005253663322651083,
 -0.009874355115762394,
 -0.026171704933991537,
 0.020348368352341755,
 0.02257376179702522,
 -0.00752236652698065,
 0.017230149476854816,
 -0.005986577872327018,
 0.01914905397335298,
 0.021254515421344024,
 -0.015644390243455687,
 0.007642298058011785,
 -0.018402813128865346,
 -0.0006866907980891692,
 -0.006416332020722187,
 -0.010967063792870444,
 0.0180030410483207,
 -0.022173991579125737,
 -0.0030682459684349844,
 0.014405098842676966,
 -0.029982859474719296,
 0.018496092535934365,
 -0.007935463505353126,
 0.01004758949773389,
 -0.019442219420694323,
 -0.003694554583399551,
 -0.024279453027116777,
 -0.005540166088247863,
 0.0038311432844533796,
 -0.00566676076668485,
 -0.029449831896638266,
 0.018576046579514262,
 -0.016643818582172135,
 0.007529029208725211,
 0.012452880937455994,
 -0.00097111

In [36]:
doc_result = embeddings.embed_documents([text])
doc_result

[[-0.0031584087512342572,
  0.011094410212838685,
  -0.004001317166816525,
  -0.011747414500761147,
  -0.0010153218392504927,
  0.010781234363886399,
  -0.010368109905383128,
  -0.0052973312542353886,
  -0.00988168756470157,
  -0.026160153195893116,
  0.02037639960097617,
  0.022575293225485345,
  -0.007522876537439858,
  0.01728462465309313,
  -0.0060036416535162295,
  0.01912369806781222,
  0.02125595742330888,
  -0.015645450320121206,
  0.007669469042167345,
  -0.018364081557173,
  -0.0006909018997805782,
  -0.006416767043342096,
  -0.010981134376956157,
  0.017950956167347133,
  -0.022135513382996398,
  -0.0030767832152439494,
  0.014379421560625712,
  -0.029984892064397486,
  0.018524000822570733,
  -0.007916012019091008,
  0.010068260351439788,
  -0.019456863824939222,
  -0.003688141783525537,
  -0.02424112014847516,
  -0.005530547004827511,
  0.0038247395669963142,
  -0.005653818260458694,
  -0.029451827225523322,
  0.018603960455269596,
  -0.016618293138839127,
  0.007529539684

下面这个例子是使用HuggingFace Hub的embeddings模型，让我们看看它的一个代码样例：

In [45]:
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
#使用前需要保证库中有sentence_transformers
%pip install sentence_transformers

于此同时将会下载安装相关依赖文件以及众多json格式的解析处理文件，安装之后就可以正常使用HuggingFaceEmbeddings了

In [53]:
embeddings_hf = HuggingFaceEmbeddings()

In [50]:
text_hf = "I just test the embeddings method in huggingface"

In [54]:
query_result = embeddings_hf.embed_query(text_hf)
query_result

[-0.02886896708001464,
 0.0044634718471356595,
 0.006389421280895015,
 -0.009913570977266759,
 -0.0027808003958931854,
 0.023057329930755516,
 0.018016073486322687,
 -0.007974105724918213,
 -0.02111110793326557,
 -0.030896281195072053,
 -0.003899202602766403,
 0.009474319213141963,
 -0.002067861337994345,
 -0.0010989737706240251,
 -0.008920186017807362,
 0.00898100603730552,
 0.01804310419217851,
 0.010190636643611502,
 0.007784889852604894,
 -0.026733528278888818,
 0.019691986562609287,
 -0.01051500790784905,
 -0.010170363614219635,
 -0.010170363614219635,
 -0.016096882194688564,
 0.011109686230644828,
 0.03597808556093373,
 -0.027652577865922144,
 -0.006450240834731895,
 -0.009197252615474662,
 0.014083082501236506,
 -0.012393653140699437,
 -0.005676482222996175,
 -0.025692840515504287,
 -0.014461514245863142,
 -0.002672677106808614,
 0.0069536905252642715,
 -0.025300891555304628,
 0.025963147574062522,
 -0.0059940948794472105,
 -0.00814980624430362,
 0.0155832950581377,
 -0.01094074

In [55]:
doc_result = embeddings_hf.embed_documents([text_hf])
doc_result

[[-0.02886896708001464,
  0.0044634718471356595,
  0.006389421280895015,
  -0.009913570977266759,
  -0.0027808003958931854,
  0.023057329930755516,
  0.018016073486322687,
  -0.007974105724918213,
  -0.02111110793326557,
  -0.030896281195072053,
  -0.003899202602766403,
  0.009474319213141963,
  -0.002067861337994345,
  -0.0010989737706240251,
  -0.008920186017807362,
  0.00898100603730552,
  0.01804310419217851,
  0.010190636643611502,
  0.007784889852604894,
  -0.026733528278888818,
  0.019691986562609287,
  -0.01051500790784905,
  -0.010170363614219635,
  -0.010170363614219635,
  -0.016096882194688564,
  0.011109686230644828,
  0.03597808556093373,
  -0.027652577865922144,
  -0.006450240834731895,
  -0.009197252615474662,
  0.014083082501236506,
  -0.012393653140699437,
  -0.005676482222996175,
  -0.025692840515504287,
  -0.014461514245863142,
  -0.002672677106808614,
  0.0069536905252642715,
  -0.025300891555304628,
  0.025963147574062522,
  -0.0059940948794472105,
  -0.00814980624

上面给出的例子用于熟悉基本models中的相关操作，详细的API开放接口还有很多，例如针对微软Azure的AzureOpenAI，Cohere和Llama-cpp等，点击[此处](https://python.langchain.com/en/latest/modules/models/text_embedding.html)可以直接跳转到相关位置。

---

## Prompt
Prompt是一种为了更好地使用预训练模型的知识，采用在输入端添加额外文本的技术。Prompt的设计有时候可以极大地激发出大模型的潜力，基于prompt可以完成很多我们想象不到的任务，那么，现在我们就来熟悉一下什么是Prompt以及在LangChain中如何去更好地使用和完善它们，LangChain提供了几个类和函数来轻松构建Prompt模板。

- [Prompt Templates](#prompt-templates)
- [Chat Prompt Template](#chat-model-prompt-template)
- [Example Selectors](#example-selectors)
- [Output Parser](#output-parser)

### Prompt Templates
语言模型的文本输入字符串本身就是一种提示，通常来说prompt由模板、一些例子（有的话）和用户的输入构成，其概念和含义并不复杂。

### 什么是prompt template：
Prompt Template，也就是提示模版，是一种可重复使用生成prompt的方式，用template可以模版化提示指令，它由一个字符串和可供用户输入的参数组成。
提示模版包含如下的内容：
- 给语言模型的指令
- 几个例子，帮助语言模型更好地回答
- 一个让语言模型回答的问题
- ...

现在我们看一个创建prompt template的例子：

In [3]:
#一般导入的必要库，为了进一步模版化处理
from langchain import PromptTemplate

#具体的模版内容，其中要进行补全的地方（也就是为了定制我们具体问题以及要求的地方用一个变量进行代替）用中括号
#进行变量的放置，不需要其他的操作，类似于字符串中对于某一个变量对其的format格式化
template = """
I want you to act as a naming consultant for new companies.
Here are some examples of good company names:
- search engine, Google
- social media, Facebook
- video sharing, YouTube
The name should be short, catchy and easy to remember.
What is a good name for a company that makes {product}?
"""

"""input_variables就是模版中需要填补的变量，在prompttemplate中格式化模版可以将变量以及模版内容进行很好的组合"""
prompt = PromptTemplate(
    input_variables=["product"],
    template=template,
)

让我们看看prompt长什么样子：

In [4]:
prompt

PromptTemplate(input_variables=['product'], output_parser=None, partial_variables={}, template='\nI want you to act as a naming consultant for new companies.\nHere are some examples of good company names:\n- search engine, Google\n- social media, Facebook\n- video sharing, YouTube\nThe name should be short, catchy and easy to remember.\nWhat is a good name for a company that makes {product}?\n', template_format='f-string', validate_template=True)

### 创建prompt template
我们可以从上面的示例中看到：通过LangChain的PromptTemplate类，可以生成具体的prompt，那么通过继续使用这个类生成几个具体的prompts。prompt template接受多个输入变量，用来格式化生成prompt

In [6]:
from langchain import PromptTemplate

# input_variables的值可以为空，说明其中没有任何变量
no_input_prompt = PromptTemplate(input_variables=[], template="Tell me a joke.")
no_input_prompt.format()

'Tell me a joke.'

In [7]:
# 一个input_variable的示例，这样模版化之后的提示将把adjective作为参数传入
one_input_prompt = PromptTemplate(input_variables=["adjective"], template="Tell me a {adjective} joke.")
one_input_prompt.format(adjective="funny")

'Tell me a funny joke.'

In [8]:
# 多个input_variables的示例，模版后的提示将adjective和content作为参数传入
multiple_input_prompt = PromptTemplate(
    input_variables=["adjective", "content"],
    template="Tell me a {adjective} joke about {content}."
)
multiple_input_prompt.format(adjective="funny", content="chickens")

'Tell me a funny joke about chickens.'

### 从LangChainHub加载本地prompt template

In [11]:
from langchain.prompts import load_prompt
#可以加载用json格式写好的prompt
test_prompt = PromptTemplate(
    input_variables=["input"],
    template="{input}, tell me the answer by using Chinese."
)
#将创建好的提示模版格式化，其中的补全的变量就是1+1等于几？
test_prompt.format(input="what is 1+1?")
#然后将创建好的模版保存在指定位置处
test_prompt.save("test_prompt.json")

In [13]:
#加载本地的模版使用到的方法是固定的，将具体文件的位置传入即可
prompt = load_prompt("./test_prompt.json")
prompt

PromptTemplate(input_variables=['input'], output_parser=None, partial_variables={}, template='{input}, tell me the answer by using Chinese.', template_format='f-string', validate_template=True)

### 在prompt template中添加几个例子
如果生成带有几个例子的模版，该怎么做？通过PromptTemplate类固然是可以，将例子固定在具体的模版中是我们想到的方法。那么LangChain有没有提供直接的类可以用来达到我们的目的和要求。下面是给语言模型几个合适的例子从而使大模型能够更准确、更合适地回答问题，LangChain中使用FewShotPromptTemplate类生成带有例子的prompt。

下面是创建一个让大语言模型去回答反义词的提示模版：

In [126]:
from langchain import PromptTemplate, FewShotPromptTemplate
#如下是几个提示示例，让大模型知道几个例子回答的时候可以类比着回答
few_examples = [
    {"word": "happy", "antonym": "sad"},
    {"word": "tall", "antonym": "short"},
]

#我们定义如下的提示模版
example_formatter_template = """
Word: {word}
Antonym: {antonym}\n
"""
#模版格式化提示模版
example_prompt = PromptTemplate(
    input_variables=["word", "antonym"],
    template=example_formatter_template,
)

#现在构建一个少样本提示模版对象：few_shot_prompt
'''
其中的参数有examples, example_prompt, prefix, suffix, input_variables和example_separator。其中，
    examples:示例，样例，这个就是我们写好的几个供大模型了解格式以及例子的具体样例
    example_prompt:具体的模版化后的提示模版，这个可以就是通过promptTemplate构建的提示模版，只等待填入具体的变量，
其格式内容中已经将待补全的变量放在其中
    prefix:前缀，一般作为大模型接受指令的入口，可以去声明大模型即将承担的角色以及告诉大模型即将要做的事情等
    suffix:后缀，一般放置于样例之后，作为提示模版的最后一部分
    input_variables:填补的变量
    example_separator:分割符，分割前缀、样例和后缀的标志
'''
few_shot_prompt = FewShotPromptTemplate(
    examples=few_examples,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",
    suffix="Word: {input}\nAntonym:",
    input_variables=["input"],
    example_separator="\n\n",
)

print(few_shot_prompt.format(input="big"))

Give the antonym of every input


Word: happy
Antonym: sad




Word: tall
Antonym: short



Word: big
Antonym:


### 从prompt template中选取例子
上面那个代码案例中提供的是少量样本示例，如果有非常多可供LLM参考的例子时，使用ExampleSelector类来可控制地选择几个最好的例子供LLM学习。

下面是一个使用LengthBasedExampleSelector选择一定长输入的例子，它是一个自动选择例子的方法，我们具体来看：**如果用户担心prompt超过输入窗口大小时，这很有用。当用户输入很长时，它自动选择少量的例子；当用户输入很短时，它选择更多的例子。**

In [127]:
from langchain.prompts.example_selector import LengthBasedExampleSelector
#依然和上面一样，反义词对儿的组合，这里给出下面几种：
few_examples = [
    {"word": "beautiful", "antonym": "ugly"},
    {"word": "outgoing", "antonym": "incoming"},
    {"word": "happy", "antonym": "sad"},
    {"word": "tall", "antonym": "short"},
    {"word": "energetic", "antonym": "lethargic"},
    {"word": "sunny", "antonym": "gloomy"},
    {"word": "windy", "antonym": "calm"},
]
#我们定义如下的提示模版
example_formatter_template = """
Word: {word}
Antonym: {antonym}\n
"""
#模版格式化提示模版
example_prompt = PromptTemplate(
    input_variables=["word", "antonym"],
    template=example_formatter_template,
)

'''
使用LengthBasedExampleSelector选择例子。
其中的examples:同上示例中的作用
example_prompt:同上示例中的作用
max_length:这是格式化后的例子的最大长度。长度由下面的get_text_length函数决定。
'''
example_selector = LengthBasedExampleSelector(
    examples=few_examples,
    example_prompt=example_prompt,
    max_length=25,
)

#现在使用example_selector来创建fewshotprompttemplate
dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",
    suffix="Word: {input}\nAntonym:",
    input_variables=["input"],
    example_separator="\n\n",
)

print(dynamic_prompt.format(input="big"))

Give the antonym of every input


Word: happy
Antonym: sad




Word: tall
Antonym: short




Word: energetic
Antonym: lethargic



Word: big
Antonym:


根据上面的例子我们知道：当输入问题很长时，LengthBasedExampleSelector会选择更少的提示

In [128]:
long_string = "big and huge and massive and large and gigantic and tall and much much much much much bigger than everything else"
print(dynamic_prompt.format(input=long_string))

Give the antonym of every input

Word: big and huge and massive and large and gigantic and tall and much much much much much bigger than everything else
Antonym:


### chat model prompt template
下面的示例是利用prompt template在chat model中使用的情况：chat model可以使用以前的历史信息进行单次生成回复，单次输入包含了过去聊天中的一系列模板、例子、用户问题的组合。LangChain提供了一些类和方法使得构建和使用prompt更加容易。

In [13]:
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

ChatPromptTemplate可以使用一个或者多个MessagePromptTemplate类构建prompt。可以使用ChatPromptTemplate的format_prompt函数返回prompt值，然后将其转化为string或message对象。

In [14]:
system_template="You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [131]:
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
# 从格式化的信息中获取完整的聊天信息
chat_prompt.format_prompt(input_language="English", output_language="French", text="I love programming.").to_messages()

[SystemMessage(content='You are a helpful assistant that translates English to French.', additional_kwargs={}),
 HumanMessage(content='I love programming.', additional_kwargs={})]

下面是一种快速构建MessagePromptTemplate的方法

In [15]:
prompt=PromptTemplate(
    template="You are a helpful assistant that translates {input_language} to {output_language}.",
    input_variables=["input_language", "output_language"],
)
system_message_prompt = SystemMessagePromptTemplate(prompt=prompt)

### Example Selectors
根据模型功能需要动态选择提示词
LangChain中的BaseExampleSelector类用于选择例子，select_examples函数接收输入变量并返回一系列例子。

### 基本接口定义如下

In [133]:
from abc import ABC, abstractmethod
from typing import Dict,List
'''它是一个抽象基类，不能被示例化，而是用于定义其他类的接口和规范'''
class BaseExampleSelector(ABC):
    """
    这是一个抽象方法，由派生类实现，以满足BaseExampleSelector接口的要求，这个方法是根据输入变量选择要包含在提示中的示例，返回一个字典列表
    """
    @abstractmethod
    def select_examples(self, input_variables: Dict[str, str]) -> List[dict]:

一个ExampleSelector类必须实现两个方法：
- 1. add_example方法接受一个例子将其添加到example列表中
- 2. select_examples告诉选择器如何选择例子并返回例子

因此在一个示例选择器中可以随时调用上述两种方法，下面实现一个自定义的example selector

In [134]:
from langchain.prompts.example_selector.base import BaseExampleSelector
from typing import Dict, List
import numpy as np

class CustomExampleSelector(BaseExampleSelector):

    def __init__(self, examples: List[Dict[str, str]]):
        self.examples = examples

    def add_example(self, example: Dict[str, str]) -> None:
        """添加新的例子来存储一个键"""
        self.examples.append(example)

    def select_examples(self, input_variables: Dict[str, str]) -> List[dict]:
        """根据输入的信息选择要使用的例子"""
        return np.random.choice(self.examples, size=2, replace=False)


使用自定义的example selector

In [135]:

examples = [
    {"foo": "1"},
    {"foo": "2"},
    {"foo": "3"}
]

# 初始化示例选择器
example_selector = CustomExampleSelector(examples)
# 选择示例
example_selector.select_examples({"foo": "foo"})
# 将新的例子添加到例子集中
example_selector.add_example({"foo": "4"})
example_selector.examples
# 选择示例
example_selector.select_examples({"foo": "foo"})

array([{'foo': '1'}, {'foo': '3'}], dtype=object)

### 基于长度的示例选择器
LengthBased ExampleSelector根据用户输入自动选择一定数量的示例；使总长度不超过LLM输入窗口大小。下面的示例我们上面就遇到过，现在我们走到这里再看一下这个代码示例，是否更加了解它的原理？

In [136]:
from langchain.prompts import PromptTemplate
from langchain.prompts import FewShotPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector
# 这些是很多关于创造反义词的例子。
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
]

In [137]:
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)
example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=25,
)
dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",
    suffix="Input: {adjective}\nOutput:",
    input_variables=["adjective"],
)

In [138]:
print(dynamic_prompt.format(adjective="big"))

Give the antonym of every input

Input: happy
Output: sad

Input: tall
Output: short

Input: energetic
Output: lethargic

Input: sunny
Output: gloomy

Input: windy
Output: calm

Input: big
Output:


In [139]:
long_string = "big and huge and massive and large and gigantic and tall and much much much much much bigger than everything else"
print(dynamic_prompt.format(adjective=long_string))

Give the antonym of every input

Input: happy
Output: sad

Input: big and huge and massive and large and gigantic and tall and much much much much much bigger than everything else
Output:


In [140]:
# You can add an example to an example selector as well.
new_example = {"input": "big", "output": "small"}
dynamic_prompt.example_selector.add_example(new_example)
print(dynamic_prompt.format(adjective="enthusiastic"))

Give the antonym of every input

Input: happy
Output: sad

Input: tall
Output: short

Input: energetic
Output: lethargic

Input: sunny
Output: gloomy

Input: windy
Output: calm

Input: big
Output: small

Input: enthusiastic
Output:


### Similarity Example Selector
The Similarity Example Selector selects examples based on their similarity to the input. This selector works by calculating the cosine similarity between the example and the word embedding vectors of the input.

In [141]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)

# These are many examples of creating antonyms.
few_examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
]

In [142]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    few_examples,
    OpenAIEmbeddings(),
    Chroma,
    k=1
)
similar_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",
    suffix="Input: {adjective}\nOutput:",
    input_variables=["adjective"],
)

Using embedded DuckDB without persistence: data will be transient


In [143]:
print(similar_prompt.format(adjective="worried"))

Give the antonym of every input

Input: happy
Output: sad

Input: worried
Output:


In [144]:
print(similar_prompt.format(adjective="fat"))

Give the antonym of every input

Input: happy
Output: sad

Input: fat
Output:


In [145]:
similar_prompt.example_selector.add_example({"input": "enthusiastic", "output": "apathetic"})
print(similar_prompt.format(adjective="joyful"))

Give the antonym of every input

Input: happy
Output: sad

Input: joyful
Output:


### Output Parser
Output parsers, as their name implies, are used to parse the output answers. They instruct the model to format the output content and parse it into the desired format. The class structure of output parsers helps to structure the response information from the model. For example, if we define in advance that the output should include properties A and B, when we send the question to the large model, the model will not specifically divide the answer for you. Therefore, we need to parse this answer and provide it to users so that they can use the parsed results for more convenient applications or achieve their goals.

Below is an example of PydanticOutputParser, which allows the LLM output to be in JSON format. The effectiveness of this feature is related to the capabilities of the LLM to generate outputs.

Below is an example of using PydanticOutputParser.

In [146]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List

In [147]:
model_name = 'text-davinci-003'
temperature = 0.0
model = OpenAI(model_name=model_name, temperature=temperature)

In [148]:
# Define the data structure you want
"""
The following code defines a Pydantic model class called Joke, 
which is used to represent a joke. After instantiation, it is used to 
prompt the large model to fill this data structure.
"""
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")
    @validator('setup')
    def question_ends_with_question_mark(cls, field):
        if field[-1] != '?':
            raise ValueError("Badly formed question!")
        return field

In [149]:
# The parameter value is the previously defined joke class, and the parser object's role is to parse the output results into a joke object
parser = PydanticOutputParser(pydantic_object=Joke)

In [150]:
'''
The two placeholders for template parameters in prompttemplate are format_instructions 
and query. However, the specified input_variables is query, while partial_variables is 
a dictionary that contains variable information with keys as format_instructions. This 
value is obtained by calling the get_format_instructions method on the previously defined 
parser object and returning a format specification string.
'''
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

In [151]:
joke_query = "Tell me a joke."
_input = prompt.format_prompt(query=joke_query)

In [152]:
output = model(_input.to_string())

In [153]:
parser.parse(output)

Joke(setup='Why did the chicken cross the road?', punchline='To get to the other side!')

Here is another example. Let's continue to look at it.

In [154]:
"""
The following code defines a data structure called actor, which is also created based on the basemodel class. This actor has two attributes, name and file_names, which represent the actor's name and the list of movies they have acted in. These two attributes are also subject to the field class, with description attributes assigned to each of them to describe their roles.
Here are some details:
actor_query: The user's question
parser: An instance of the PydanticOutputParser class, which is used to parse the output results into an actor object
prompt: An instance of the prompttemplate class, which contains a string template and some variables
"""
class Actor(BaseModel):
    name: str = Field(description="name of an actor")
    film_names: List[str] = Field(description="list of names of films they starred in")

actor_query = "Generate the filmography for a random actor."

parser = PydanticOutputParser(pydantic_object=Actor)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

_input = prompt.format_prompt(query=actor_query)

output = model(_input.to_string())

parser.parse(output)

Actor(name='Tom Hanks', film_names=['Forrest Gump', 'Saving Private Ryan', 'The Green Mile', 'Cast Away', 'Toy Story'])

---

## Index
This module is used to accept user queries and return the most relevant concepts.
Here, LangChain mainly builds an index for the user-provided documents, combining them into a searcher; then it can establish a question-answering chain for model questioning and answering.
LangChain uses chromadb to build a vector pool called vectorstore, which is used for retrieval and word embedding lookups.
For more detailed documentation, click 👉[here](https://python.langchain.com/en/latest/modules/indexes.html) 👈 directly.

Understanding basic index concepts is also important. The following is an interface for the indexer Retriever, and users can implement how to return relevant documents on their own. LangChain focuses on using the Vectorstore retriever for retrieving related documents.

In [155]:
from abc import ABC, abstractmethod
from typing import List
from langchain.schema import Document

class BaseRetriever(ABC):
    @abstractmethod
    def get_relevant_documents(self, query: str) -> List[Document]:
        """Get texts relevant for a query.

        Args:
            query: string to find relevant texts for

        Returns:
            List of relevant documents
        """

In [156]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.document_loaders import TextLoader
loader = TextLoader('state_of_the_union.txt',encoding='utf-8')

create the index

Using the VectorstoreIndexCreator to create index directly

In [157]:
"""
The VectorstoreIndexCreator class is used to create a vector space index
(Vector Space Index). To create an object named index, call the from_loaders 
method with a parameter list containing only one element, which is the TextLoader 
object loader mentioned earlier. The purpose of from_loaders is to package multiple 
text data loaders (TextLoader) into a vector space index object. 
Therefore, the role of index is to read the contents of the state_of_the_union.txt 
file into memory and store it in the variable loader as a string, and then use the 
VectorstoreIndexCreator class to package it into an index object called index.
"""
from langchain.indexes import VectorstoreIndexCreator
index = VectorstoreIndexCreator().from_loaders([loader])

Using embedded DuckDB without persistence: data will be transient


we constructed the index along with you can answer the queston by providing docs

In [158]:
query = "What did the president say about Ketanji Brown Jackson"
index.query(query)

" The president said that Ketanji Brown Jackson is one of the nation's top legal minds, a former top litigator in private practice, a former federal public defender, and from a family of public school educators and police officers. He also said that she is a consensus builder and has received a broad range of support from the Fraternal Order of Police to former judges appointed by Democrats and Republicans."

In [159]:
query = "What did the president say about Ketanji Brown Jackson"
index.query_with_sources(query)

{'question': 'What did the president say about Ketanji Brown Jackson',
 'answer': " The president said that he nominated Circuit Court of Appeals Judge Ketanji Brown Jackson, one of the nation's top legal minds, to continue Justice Breyer's legacy of excellence, and that she has received a broad range of support from the Fraternal Order of Police to former judges appointed by Democrats and Republicans.\n",
 'sources': 'state_of_the_union.txt'}

if you only see the vectorstore, you can use following coding :

In [160]:
index.vectorstore

or maybe you wanna see the format of VectorstoreRetriever

In [161]:
index.vectorstore.as_retriever()

VectorStoreRetriever(vectorstore=<langchain.vectorstores.chroma.Chroma object at 0x0000018FA13C05B0>, search_type='similarity', search_kwargs={})

The above example shows how to use an index for querying. So, how is the index created? There are three steps:

- Divide the documents into chunks
- Create word embeddings for each chunk
- Store both the documents and word embeddings in a vector pool

load the docs:

In [162]:
documents = loader.load()

split the docs

In [163]:
from langchain.text_splitter import CharacterTextSplitter
"""
The CharacterTextSplitter class is used to chunk text data according 
to a specified size. Next, the code creates an object named text_splitter 
by calling the split_documents method, which takes two parameters: 
chunk_size, which indicates the size of each document chunk; and chunk_overlap, 
which indicates the overlap size between documents. The chunk_size is set to 1000, 
and the chunk_overlap is set to 0. Then, the code calls the split_documents method 
on the documents list to split the documents into chunks of the specified size and 
returns a list containing all the split documents. Specifically, the split_documents 
method splits a list of texts into chunks of a specified size and returns a list 
containing all the split documents.
"""
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

Select an appropriate word embedding method as needed for word embedding

In [164]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

Next, use word embeddings and chunk creation to create a vector pool

In [165]:
from langchain.vectorstores import Chroma
"""
The Chroma class is used to combine a Vector Space Index (Vector Space Index) with text data
to achieve a vectorized representation of text data. Next, the code creates an object named 
db by calling the from_documents method, which takes two parameters: texts and embeddings. 
Among them, texts is a list containing all text data, and embeddings is a two-dimensional 
array containing all vector data. The purpose of from_documents is to combine the specified 
text data and vector data and return a two-dimensional array containing all document vectors. 
Specifically, from_documents combines the specified text data and vector data to achieve a 
vectorized representation of text data.
"""
db = Chroma.from_documents(texts, embeddings)

Using embedded DuckDB without persistence: data will be transient


then, construct the index

In [166]:
"""
The from_documents method of Chroma class is used to combine text data and vector 
data to create a document vector representation of the text data. The resulting 
document vectors are stored in a two-dimensional array called embeddings.
To retrieve relevant document vectors for a given query, we can use the as_retriever 
method of the db object. This method converts the db object into a retriever object, 
which can be used to search for document vectors that match the query
"""
retriever = db.as_retriever()

next, you can construct a chain to answer the question:

In [167]:
"""
The RetrievalQA class is used to implement the Question Answering (QA) task based on 
the retriever. Next, the code creates an object named qa by calling the from_chain_type 
method, which takes three parameters: llm, chain_type, and retriever. Among them, llm 
represents a language model (Language Model), chain_type represents the query type, and 
retriever represents a retriever (retriever). The purpose of from_chain_type is to create 
an object named qa based on the specified query type and retriever for later use in calculations. The llm is set to OpenAI(), chain_type is set to "stuff", and retriever is set to the previously created retriever object.
"""
qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=retriever)
query = "What did the president say about Ketanji Brown Jackson"
"""
The run method performs the following operations: First, it retrieves document vectors 
related to the given query from the database; then, it matches the user input question 
with each document vector and finds the best document vector as the answer; finally, it 
encapsulates the answer into a triple and returns it to the user.
"""
qa.run(query)

" The President said that Ketanji Brown Jackson is one of the nation's top legal minds and that she will continue Justice Breyer's legacy of excellence."

To summarize the above process, VectorstoreIndexCreator performs the steps of chunking, word embedding, and index creation.

In [168]:
index_creator = VectorstoreIndexCreator(
    vectorstore_cls=Chroma,
    embedding=OpenAIEmbeddings(),
    text_splitter=CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
)

---

## Memory
Memory involves maintaining state concepts throughout the entire interaction between the user and the language model. The interaction between the user and the language model is captured in the concept of ChatMessages, which thus boils down to capturing, capturing, transforming, and extracting knowledge from a series of chat messages. There are many different methods to achieve this, each with its own type of memory. Typically, there are two ways to understand the use of memory for each type. These are independent functions that extract information from a series of messages, and then you can use this type of memory in your chain. Memory can return multiple pieces of information (e.g., the N most recent messages and a summary of all previous messages). The returned information can be strings or message lists. We will introduce the simplest form of memory: "Buffer" memory, which only involves keeping a buffer for all previous messages. We will demonstrate here how to use modular utility functions and show how to use it in chains (both returning strings and message lists).

LLMs and chat models are stateless; every input request is independent; Chains and Agents are developed based on underlying modules and are also stateless. In some applications, such as chatbots, it is important for the language model to know about the previous chat content. This is why the Memory module exists.

LangChain provides two types of memory components; first, LangChain provides helper utilities to manage and operate on previous chat information; second, LangChain provides methods to merge these programs into Chains modules.

To view the full documentation and example instructions, click 👉[LangChain Official Website](https://python.langchain.com/en/latest/index.html).

The following are two examples of adding Memory to quickly understand:

- [Add Memory to an LLMChain](#add-memory-to-an-llmchain)
- [Add Memory to an Agent](#add-memory-to-an-agent)


We will use the ConversationBufferMemory as an example, so let's take a look at what this class looks like first:

### ConversationBufferMemory
The ConversationBufferMemory can help users easily create a conversation history, as follows:

In [5]:
from langchain.memory import ConversationBufferMemory

"""
The ConversationBufferMemory class is used to implement a conversational 
system based on memory caches (Memories). Next, the code creates an object 
named memory by calling the from_memory method without any parameters. The 
purpose of this method is to create an object named memory based on default 
configuration for later use in calculations. Memory will be used to store the 
conversation history between the user and AI. Then, the chat_memory attribute 
of the memory object is called to create a conversation buffer (Conversation Buffer) 
and add two messages: one from the user saying "hi!" and the other from the AI asking 
"what's up?". Specifically, the chat_memory.add_user_message() method adds a user message
 to the conversation buffer, while the chat_memory.add_ai_message() method adds an AI 
 message to the conversation buffer. In this way, the chat history has been successfully 
 added to the conversation buffer.
"""
memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("hi!")
memory.chat_memory.add_ai_message("whats up?")

In [6]:
# Loading variables stored in the conversation buffer. The method takes an empty 
# dictionary as a parameter, indicating that no variables need to be loaded.
memory.load_memory_variables({})

{'history': 'Human: hi!\nAI: whats up?'}

list can return the history news

In [11]:
memory = ConversationBufferMemory(return_messages=True)
memory.chat_memory.add_user_message("hi!")
memory.chat_memory.add_ai_message("whats up?")

In [12]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='hi!', additional_kwargs={}),
  AIMessage(content='whats up?', additional_kwargs={})]}

This is a simple and direct way of recording and obtaining historical conversations. Next, let's take a look at how to add them to the Chain through several examples:

### Add Memory to an LLMChain

The following shows how to add ConversationBufferMemory. For other types of Memory, you can click [here](https://python.langchain.com/en/latest/modules/memory/how_to_guides.html) to view and use them.

In [13]:
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain import LLMChain, PromptTemplate

template = """You are a chatbot having a conversation with a human.
{chat_history}
Human: {human_input}
Chatbot:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input"], 
    template=template
)
memory = ConversationBufferMemory(memory_key="chat_history")

llm_chain = LLMChain(
    llm=OpenAI(), 
    prompt=prompt, 
    verbose=True, 
    memory=memory,
)

In [14]:
llm_chain.predict(human_input="Hi there!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there!
AI:

> Finished chain.


" Hi there! It's nice to meet you. My name is AI. What's your name?"

In [15]:
llm_chain.predict(human_input="I'm doing well! Just having a conversation with an AI.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI:  Hi there! It's nice to meet you. My name is AI. What's your name?
Human: I'm doing well! Just having a conversation with an AI.
AI:

> Finished chain.


" That's great! It's always nice to have a conversation with someone new. What would you like to talk about?"

In [16]:
llm_chain.predict(input="Tell me about yourself.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI:  Hi there! It's nice to meet you. My name is AI. What's your name?
Human: I'm doing well! Just having a conversation with an AI.
AI:  That's great! It's always nice to have a conversation with someone new. What would you like to talk about?
Human: Tell me about yourself.
AI:

> Finished chain.


" Sure! I'm an AI created to help people with their everyday tasks. I'm programmed to understand natural language and provide helpful information. I'm also able to learn from my conversations and experiences, so I'm constantly growing and evolving. What else would you like to know?"

In another scenario, users need to save historical information because they have the need to load and reuse it when needed. The schema class of LangChain can conveniently convert historical information into Python data structures such as dictionaries; it can also be converted into JSON format; and then load historical information from dictionaries or JSON. As shown below:

In [17]:
import json

from langchain.memory import ChatMessageHistory
from langchain.schema import messages_from_dict, messages_to_dict

history = ChatMessageHistory()
history.add_user_message("hi!")
history.add_ai_message("whats up?")

In [18]:
dicts = messages_to_dict(history.messages)

In [19]:
dicts

[{'type': 'human', 'data': {'content': 'hi!', 'additional_kwargs': {}}},
 {'type': 'ai', 'data': {'content': 'whats up?', 'additional_kwargs': {}}}]

In [20]:
new_messages = messages_from_dict(dicts)

In [21]:
new_messages

[HumanMessage(content='hi!', additional_kwargs={}),
 AIMessage(content='whats up?', additional_kwargs={})]

The following is a case study on adding Memory to an Agent:

### Add Memory to an Agent


In order to add Memory to an Agent, we will follow these steps:

- We will create an LLMChain with Memory.
- We will use this LLMChain to create a custom agent.
- We will create a simple custom agent that can access search tools and use the ConversationBufferMemory class.

In [ ]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain import OpenAI, LLMChain
from langchain.utilities import GoogleSearchAPIWrapper

search = GoogleSearchAPIWrapper()
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    )
]


prefix = """Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:"""
suffix = """Begin!"

{chat_history}
Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools, 
    prefix=prefix, 
    suffix=suffix, 
    input_variables=["input", "chat_history", "agent_scratchpad"]
)
memory = ConversationBufferMemory(memory_key="chat_history")

llm_chain = LLMChain(llm=OpenAI(temperature=0), prompt=prompt)
agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)
agent_chain = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True, memory=memory)

agent_chain.run(input="How many people live in canada?")

To test the agent's memory, we can ask a follow-up question that depends on the information from previous conversations in order to provide an accurate answer.

In [ ]:
agent_chain.run(input="what is their national anthem called?")

We will compare it with an agent without memory.

In [ ]:
prefix = """Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:"""
suffix = """Begin!"

Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools, 
    prefix=prefix, 
    suffix=suffix, 
    input_variables=["input", "agent_scratchpad"]
)
llm_chain = LLMChain(llm=OpenAI(temperature=0), prompt=prompt)
agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)
agent_without_memory = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)
agent_without_memory.run("How many people live in canada?")

In [ ]:
agent_without_memory.run("what is their national anthem called?")

Of course, the above operation examples are for readers to quickly understand and get started. In order to apply more proficiently in development, it is still necessary to have a comprehensive review of official tutorials for further integration. Click [here](https://python.langchain.com/en/latest/modules/memory.html) to go directly to the Memory tutorial location.

---

## Chains
When implementing some simple applications, it is very feasible to use only one large language model. However, in some complex cases, we need to combine different large language models or combine large language models with other modules to complete some huge and complex work. LangChain provides a standard interface and some commonly used implementation methods for Chains.

For example, as we have learned before - we can create a chain that receives user input, formats it using the Prompt module, and then passes the formatted content to the LLM; In complex scenarios such as combining multiple chains, we can combine multiple chains together or add other modules within a chain to work collaboratively.

This tutorial covers the following topics:

- Using a simple LLM chain
- Creating serialized links
- Creating custom chains

If you want to learn more, you can click [here](https://python.langchain.com/en/latest/modules/chains.html) to go directly to the official Chains tutorial of LangChain.

- [LLMChain Basic Usage](#llmchain-usage)
- [Using SequentialChain to combine multiple chain](#using-sequentialchain-to-combine-with-multiple-chains)
- [Customize the Chain](#customize-the-chain-class)

### LLMChain usage

LLMChain is the simplest and most commonly used chain. It receives a prompt template, formats the input information using this template, and returns the user's query response, which is what we want. Here is an example of its use:

In [23]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
llm = OpenAI(temperature=0.9)
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

In [24]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

print(chain.run("colorful socks"))



Brightly Socks!


To complete the interaction process of the chatbot using the chat model in LLMChain, here is an example:

In [25]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)
human_message_prompt = HumanMessagePromptTemplate(
        prompt=PromptTemplate(
            template="What is a good name for a company that makes {product}?",
            input_variables=["product"],
        )
    )
chat_prompt_template = ChatPromptTemplate.from_messages([human_message_prompt])
chat = ChatOpenAI(temperature=0.9)
chain = LLMChain(llm=chat, prompt=chat_prompt_template)

RainbowSocks


In [27]:
print(chain.run("colorful socks"))

Rainbow Socks Co.


### Using SequentialChain to combine with multiple chains

The SequentialChain can combine multiple chains. The SequentialChain parameter takes a list of chains as input and executes each chain in sequence, passing the return value of the first chain to the second chain, and so on. Here is an example of its use:

In [28]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)
human_message_prompt = HumanMessagePromptTemplate(
        prompt=PromptTemplate(
            template="What is a good name for a company that makes {product}?",
            input_variables=["product"],
        )
    )
second_prompt = PromptTemplate(
    input_variables=["company_name"],
    template="Write a catchphrase for the following company: {company_name}",
)
chat_prompt_template = ChatPromptTemplate.from_messages([human_message_prompt])
chat = ChatOpenAI(temperature=0.9)
chain_one = LLMChain(llm=chat, prompt=chat_prompt_template)
chain_two = LLMChain(llm=llm, prompt=second_prompt)

The next way to combine two simple LLMChains together is to use the SimpleSequentialChain to complete, ultimately achieving the process of naming the company first, and then giving this already named company a slogan.

In [29]:
from langchain.chains import SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[chain, chain_two], verbose=True)
catchphrase = overall_chain.run("colorful socks")
print(catchphrase)



> Entering new SimpleSequentialChain chain...
Rainbow Sock Co.


"Walk on the wild side with Rainbow Socks!"

> Finished chain.


"Walk on the wild side with Rainbow Socks!"


### Customize the Chain class
Using already-packaged LangChains is certainly not a problem because they provide many pre-built chains that can be used out of the box. However, sometimes users may want to create their own custom classes for special purposes.

The process of creating a custom class follows these steps:

Inherit from the Chain class
Fill in the input_keys and output_keys attributes
Implement the private method _call() that demonstrates how to execute the chain
Here is the method for creating a custom chain class.

Fistly, you need to define a ConcatenateChain class, which can input a query and return the results by two LLMChain simultaneously。

In [30]:
from langchain.chains import LLMChain
from langchain.chains.base import Chain
from typing import Dict, List
"""
The following code defines a class named ConcatenateChain that inherits from Chain. 
This class represents a chain that concatenates two other chains together.First, we
import the LLMChain and Chain classes from the langchain.chains module, as well as the 
Dict and List type annotations from the typing module.

In the ConcatenateChain class, there are two properties: chain_1 and chain_2, which 
represent the two chains to be connected. These properties are instances of the LLMChain type.

The class also defines two methods: input_keys() and output_keys(). The input_keys() 
method returns a set of input variables after concatenation, which is the union of the 
input variables of the two original chains; while the output_keys() method returns a 
list of strings, indicating that there is only one output variable with the name 
concat_output after concatenation.

Finally, the class implements a private method _call(), which is used to perform 
the actual concatenation operation. This method takes a dictionary-type parameter inputs, 
representing the input variables and their values.

This method first calls the run() method of each chain to obtain their output results, 
then adds them together to get a new output result, which is stored in a dictionary and 
returned.
"""

class ConcatenateChain(Chain):
    chain_1: LLMChain
    chain_2: LLMChain

    @property
    def input_keys(self) -> List[str]:
        all_input_vars = set(self.chain_1.input_keys).union(set(self.chain_2.input_keys))
        return list(all_input_vars)

    @property
    def output_keys(self) -> List[str]:
        return ['concat_output']

    def _call(self, inputs: Dict[str, str]) -> Dict[str, str]:
        output_1 = self.chain_1.run(inputs)
        output_2 = self.chain_2.run(inputs)
        return {'concat_output': output_1 + output_2}

In [31]:
"""
The prompt_1 variable is of type PromptTemplate, which represents a prompt template 
containing an input variable "product" with the template being "What is a good name 
for a company that makes {product}?".

The chain_1 variable is of type LLMChain and represents an instance of LLMChain with 
the llm parameter set to the previously defined llm object and the prompt parameter 
set to prompt_1.

The chain_2 variable is also of type LLMChain and represents another instance of LLMChain 
with the llm parameter set to the previously defined llm object and the prompt parameter 
set to prompt_2.

The concat_chain variable is of type ConcatenateChain and represents an object that connects 
two chains. The chain_1 and chain_2 properties are respectively corresponding to the two 
chain instances defined above.

The concat_output variable is of type string and represents the result obtained by calling 
the run() method on the concat_chain object. This result is obtained by concatenating the 
output results of chain_1 and chain_2.

Finally, the connection result is displayed through printing.
"""
prompt_1 = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)
chain_1 = LLMChain(llm=llm, prompt=prompt_1)

prompt_2 = PromptTemplate(
    input_variables=["product"],
    template="What is a good slogan for a company that makes {product}?",
)
chain_2 = LLMChain(llm=llm, prompt=prompt_2)

concat_chain = ConcatenateChain(chain_1=chain_1, chain_2=chain_2)
concat_output = concat_chain.run("colorful socks")
print(f"Concatenated output:\n{concat_output}")

Concatenated output:


Vivid Sockery.

"Step Into Colorful Comfort!"


---

## Agents
In practical applications, not only pre-defined chains may be required, but also hidden chains can be generated based on user input requests. The agent acts as a proxy and determines the actions to be taken, tools to be used, LLM outputs, observation results, or returns LLM results to users based on user input.

Before using agents, the following concepts need to be understood:

Tools: Functions that implement specific functions; can be Google search, database lookup, or Python interaction interfaces. The interface of a tool is a function that takes a string and returns a string.
LLM: The large language model that drives the agent.
Agent: The agent used.

The relationship between the three is as follows:

<img src="./agents.png" align=center width=100% />

LangChain will drive the agent to process based on user requirements, during which the large model will be called to complete the request. This request can further drive tools to perform specific tasks according to specific tasks, and then return the specific operation results of the tools to users. In this process, if the user does not need to call tools to complete the processing, they will directly get feedback from the large model.

For more detailed content, click 👉[here](https://python.langchain.com/en/latest/modules/agents.html)👈 to jump to the official Agents tutorial.

- [The simple example of using Agent](#a-simple-example-with-using-agent)
- [Tools](#tools)
- [Agent](#agent)
- [Toolkits](#toolkits)

### A simple example with using Agent

In [32]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI
llm = OpenAI(temperature=0)
tools = load_tools(["serpapi", "llm-math"], llm=llm)
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [46]:
agent.run("Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?")



> Entering new AgentExecutor chain...
 I need to find out who Leo DiCaprio's girlfriend is and then calculate her age raised to the 0.43 power.
Action: Search
Action Input: "Leo DiCaprio girlfriend"
Observation: Nina Agdal: 2016 to 2017 ... Leo and Nina were together for almost exactly a year until a source confirmed their breakup with a very familiar ...
Thought: I need to find out Nina Agdal's age
Action: Search
Action Input: "Nina Agdal age"
Observation: 31 years
Thought: I need to calculate 31 raised to the 0.43 power
Action: Calculator
Action Input: 31^0.43
Observation: Answer: 4.378098500976803

Thought: I now know the final answer
Final Answer: Nina Agdal is Leo DiCaprio's girlfriend and her current age raised to the 0.43 power is 4.378098500976803.

> Finished chain.


"Nina Agdal is Leo DiCaprio's girlfriend and her current age raised to the 0.43 power is 4.378098500976803."

### Tools
Tools are the way that agents interact with their external environment. Tools can be utilities, chains, other agents, etc.

Most of LangChain's tools are related to search, and here are just a few examples of unique tools.

- [Bing Search](#bing-search)
- [Google Search](#google-search)
- [Google Serper API](#google-serper-api)
- [Python REPL](#python-repl)
- [Bash](#bash)
- [Wikipedia API](#wikipedia-api)

Tools are typically loaded in the following way, and for chains and agents that are used as tools, initialization is required.

In [ ]:
from langchain.agents import load_tools
tool_names = [...]
tools = load_tools(tool_names)
llm = ...
tools = load_tools(tool_names, llm=llm)

#### Bing Search

The following example demonstrates how to use the Bing Web Search API component:

First, you need to set appropriate API keys and environment variables. Click [here](https://levelup.gitconnected.com/api-tutorial-how-to-use-bing-web-search-api-in-python-4165d5592a7e) to read the instructions and complete the configuration.


In [ ]:
import os
os.environ["BING_SUBSCRIPTION_KEY"] = ""
os.environ["BING_SEARCH_URL"] = ""

from langchain.utilities import BingSearchAPIWrapper
search = BingSearchAPIWrapper()
search.run("python")

In [ ]:
search = BingSearchAPIWrapper(k=1)
search.run("python")

Running the code below will return three items: snippet, title, and link.

Snippet: Description of the result.

Title: Title of the result.

Link: Link to the result.

In [ ]:
search = BingSearchAPIWrapper()

In [ ]:
search.results("apples", 5)

#### Google Search

The following example demonstrates how to use the Google Search component. You need to set appropriate API keys and environment variables. To set them up, create a GOOGLE_API_KEY in this [console](https://console.cloud.google.com/apis/credentials) and a GOOGLE_CSE_ID [here](https://programmablesearchengine.google.com/)

In [ ]:
import os
os.environ["GOOGLE_CSE_ID"] = ""
os.environ["GOOGLE_API_KEY"] = ""
from langchain.utilities import GoogleSearchAPIWrapper
search = GoogleSearchAPIWrapper()
search.run("Obama's first name?")

In [ ]:
"""
First, create a GoogleSearchAPIWrapper object search and pass in the parameter k=1 to 
return only the first result in the search results.
Then call the run() method of the search object and pass in the parameter "python" to 
search for information about Python on the Google search engine. Next, create another 
GoogleSearchAPIWrapper object search and pass in an empty value to clear the previous 
search results. Finally, call the results() method of the search object and pass in the 
parameters "apples" and 5 to search for information about apples on the Google search 
engine and return the top 5 results.
"""
search = GoogleSearchAPIWrapper(k=1)
search.run("python")
search = GoogleSearchAPIWrapper()
search.results("apples", 5)

#### Google Serper API

In order to better use the Google Web Search API, you need to register a free account to obtain an API_KEY. The registration location can be found by clicking [here](https://serper.dev/). Below is an example code.

In [ ]:
import os
os.environ["SERPER_API_KEY"] = ""

from langchain.utilities import GoogleSerperAPIWrapper
search = GoogleSerperAPIWrapper()
search.run("Obama's first name?")

In [ ]:
os.environ['OPENAI_API_KEY'] = "YOUR_OPENAI_KEY"
from langchain.utilities import GoogleSerperAPIWrapper
from langchain.llms.openai import OpenAI
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

"""
First, create an OpenAI LLM model with a temperature parameter set to 0.
Then create a GoogleSearchAPIWrapper object search for conducting searches on the Google 
search engine. Next, define a list of tools named "Intermediate Answer" which includes a 
function that calls the run() method of the search object and returns the search results 
as an answer. Afterwards, initialize an agent called self_ask_with_search that uses the 
tools in the list to interact with the LLM model and supports asking questions using the "self ask with search" method.
Finally, call the run() method of the self_ask_with_search object and pass in the question 
to obtain the corresponding answer.
"""
llm = OpenAI(temperature=0)
search = GoogleSerperAPIWrapper()
tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description="useful for when you need to ask with search"
    )
]

self_ask_with_search = initialize_agent(tools, llm, agent=AgentType.SELF_ASK_WITH_SEARCH, verbose=True)
self_ask_with_search.run("What is the hometown of the reigning men's U.S. Open champion?")

#### Python REPL

In [48]:
"""
First, import the PythonREPL class, which is used to execute interactive code in the Python 
interpreter. Next, create a PythonREPL object called python_repl for executing code in the 
Python interpreter. Finally, call the run() method of the python_repl object and pass in a 
simple Python statement "print(3**3)" to print the value of the number 27.
"""
from langchain.utilities import PythonREPL
python_repl = PythonREPL()
python_repl.run("print(3**3)")

'27\n'

#### Bash

In [54]:
"""
First, import the BashProcess class, which is used to execute command line operations in the Linux system.
Next, create a BashProcess object called bash for executing command line operations in the Linux system.
Finally, call the run() method of the bash object and pass in a simple command "echo 'My name is name'" 
to print the string "My name is name" on the terminal.
"""
from langchain.utilities import BashProcess
bash = BashProcess()
print(bash.run(' echo "My name is name" '))

"My name is name" 



#### Wikipedia API

Here is an example of how to use Wikipedia:

In [26]:
%pip install wikipedia

Note: you may need to restart the kernel to use updated packages.


In [7]:
from langchain.utilities import WikipediaAPIWrapper

In [5]:
wikipedia = WikipediaAPIWrapper()

In [6]:
wikipedia.run('HUNTER X HUNTER')

'Page: Hunter × Hunter\nSummary: Hunter × Hunter (stylized as HUNTER×HUNTER and pronounced "hunter hunter") is a Japanese manga series written and illustrated by Yoshihiro Togashi. It has been serialized in Shueisha\'s shōnen manga magazine Weekly Shōnen Jump since March 1998, although the manga has frequently gone on extended hiatuses since 2006. Its chapters have been collected in 37 tankōbon volumes as of November 2022. The story focuses on a young boy named Gon Freecss who discovers that his father, who left him at a young age, is actually a world-renowned Hunter, a licensed professional who specializes in fantastical pursuits such as locating rare or unidentified animal species, treasure hunting, surveying unexplored enclaves, or hunting down lawless individuals. Gon departs on a journey to become a Hunter and eventually find his father. Along the way, Gon meets various other Hunters and encounters the paranormal.\nHunter × Hunter was adapted into a 62-episode anime television ser

### Agent

We will now move on to discuss the agent-related content. The agent uses the LLM to determine what actions to take and in what order. An action can be either using tools and observing their output or returning a response to the user.

- [Type](#type)
- [Customize the Agent](#custom-agent)
- [Customize LLMs of Agent](#custom-llm-agent)
- [Self Ask With Search](#self-ask-with-search)

#### Type

As a professional translator, my goal is to translate Chinese into English. Now, please translate the following passage: [The following are the Agent types available in LangChain:

We use the ReAct framework for this.

**zero-shot-react-description**: This Agent determines which tools to use based solely on their descriptions. It can provide any number of tools, and requires descriptions for each tool.

**react-docstore**: This Agent uses ReAct to interact with a document library. It must provide two tools: a search tool and a find tool (with the same name). The search tool searches for documents, while the find tool looks up terms in the most recently found document. This Agent is equivalent to the original ReAct paper, particularly the Wikipedia example.

**self-ask-with-search**: This Agent uses an Intermediate Answer tool that should be able to find factual answers to questions. This Agent is equivalent to the original self ask with search paper, which provided a Google Search API as the tool.

**conversational-react-description**: This Agent is designed for conversational settings. Its prompts are designed to help the agent provide assistance and facilitate conversation. It uses ReAct to decide which tool to use and uses memory to remember previous interactions in a conversation.

#### Custom Agent

How to construct Agent：

In [ ]:
from langchain.agents import Tool,AgentExecutor
from langchain.agents import BaseSingleActionAgent
from langchain import OpenAI, SerpAPIWrapper
from typing import List, Tuple, Any, Union
from langchain.schema import AgentAction, AgentFinish
"""
First, create a SerpAPIWrapper object named search. Then, define a list called tools 
which contains a Tool object named "Search" that can be called to search for relevant 
information. Next, define a class called FakeAgent that inherits from BaseSingleActionAgent. 
The purpose of this class is to simulate a fake intelligent agent with an input_keys 
attribute that receives user input. In the plan() method, based on the current state 
and user input, the agent decides what action to take and returns an AgentAction or 
AgentFinish object. Finally, create a FakeAgent object named agent and create an AgentExecutor 
object using AgentExecutor.from_agent_and_tools(). Then call the run() method and pass in 
a query statement "How many people live in canada as of 2023?", which will be executed by 
the agent.
"""
search = SerpAPIWrapper()
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
        return_direct=True
    )
]
class FakeAgent(BaseSingleActionAgent):
    @property
    def input_keys(self):
        return ["input"]
    
    def plan(
        self, intermediate_steps: List[Tuple[AgentAction, str]], **kwargs: Any
    ) -> Union[AgentAction, AgentFinish]:
        return AgentAction(tool="Search", tool_input=kwargs["input"], log="")

    async def aplan(
        self, intermediate_steps: List[Tuple[AgentAction, str]], **kwargs: Any
    ) -> Union[AgentAction, AgentFinish]:
        return AgentAction(tool="Search", tool_input=kwargs["input"], log="")

agent = FakeAgent()
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)
agent_executor.run("How many people live in canada as of 2023?")

#### Custom LLM Agent

The LLM agent consists of several components:

- PromptTemplate: This is the prompt template that instructs the language model what to do.

- LLM: This is the language model that provides support for the agent.

- StopSequence: This instructs the LLM to stop generating output once it finds this string.

- OutputParser: This determines how the LLM output is parsed into an AgentAction or AgentFinish object.

The LLM agent is used in an AgentExecutor, which can be largely considered a loop:

- Pass user input and any previous steps to the agent (in this case, the LLMAgent)

- If the agent returns an AgentFinish, return it directly to the user.

- If the agent returns an AgentAction, use it to call a tool and obtain an observation.

- Repeat until an AgentFinish is issued by the agent.

An AgentAction consists of a response action_input, which is created by combining action and and action_input, which refers to the input of the tool used. The log can also provide additional context (which can be used for logging, tracking, etc.).

An AgentFinish is a response that contains the final message to send back to the user. It should be used to end the agent's operation.

In [ ]:
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate
from langchain import OpenAI, SerpAPIWrapper, LLMChain
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish
import re
search = SerpAPIWrapper()
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    )
]

This part instructs the agent what to do. Generally, the template should include:

- tools: The agent can choose which tools to access and how and when to call them.

- intermediate_steps: These are tuple pairs corresponding to previous (AgentAction, Observation). These are generally not directly passed to the model, but are formatted by the prompt template in a specific way before being sent to the LLM.

- input: The general user input content

In [4]:
template = """Answer the following questions as best you can, but speaking as a pirate might speak. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to speak as a pirate when giving your final answer. Use lots of "Arg"s

Question: {input}
{agent_scratchpad}"""
template

'Answer the following questions as best you can, but speaking as a pirate might speak. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin! Remember to speak as a pirate when giving your final answer. Use lots of "Arg"s\n\nQuestion: {input}\n{agent_scratchpad}'

In [ ]:
"""
This code defines a class called CustomPromptTemplate that inherits from StringPromptTemplate.

template is a string property that represents the template of the prompt message;

tools is a list property that represents the list of available tools;

format is a method that takes in parameters and formats them into a prompt message. In the 
format method, first pop a keyword parameter named intermediate_steps from the passed-in 
parameters. Then, iterate through this intermediate step list, concatenating the operation 
log and observation results of each step into a string thoughts. Next, store thoughts in 
agent_scratchpad and add the available tool information to tools and tool_names. Finally, 
call the parent class's format method and return the formatted prompt message.
"""
class CustomPromptTemplate(StringPromptTemplate):
    template: str
    tools: List[Tool]
    def format(self, **kwargs) -> str:
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        kwargs["agent_scratchpad"] = thoughts
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        return self.template.format(**kwargs)

In [ ]:
"""
This code defines a variable named prompt, of type CustomPromptTemplate. 
It is initialized by calling the constructor of the parent class StringPromptTemplate, 
passing in three parameters: template, tools, and input_variables. The template parameter 
is of string type and represents the template for the prompt message. The tools parameter 
is of list type and represents the list of available tools. The input_variables parameter 
is also of list type and represents the list of names for input variables. In this case, 
the names of the input variables are input and intermediate_steps.
"""
prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    input_variables=["input", "intermediate_steps"]
)

OutputParser is responsible for parsing the output of an LLM into AgentAction and AgentFinish. This typically depends largely on the prompts used.

At this point, you can modify the parsing to handle retries, blanks, or other scenarios.



In [ ]:
"""
A variable named output_parser of type CustomOutputParser is defined. This variable is 
initialized by calling the constructor of the parent class AgentOutputParser.

Within the CustomOutputParser class, a method named parse is defined to parse the LLM output. 
This method takes a string parameter llm_output and returns an object of type AgentAction or 
AgentFinish.

In the parse method, the input LLM output is first checked for the presence of the string 
"Final Answer:". If this string is found, it indicates that the task has been completed and 
a AgentFinish object is returned. Otherwise, regular expressions are used to extract action 
and input information from the LLM output. If either action or input information cannot be 
extracted, an exception is thrown.

Finally, an AgentAction object is created based on the extracted information and returned.
"""
class CustomOutputParser(AgentOutputParser):
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        if "Final Answer:" in llm_output:
            return AgentFinish(
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        regex = r"Action: (.*?)[\n]*Action Input:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

output_parser = CustomOutputParser()

In [ ]:
llm = OpenAI(temperature=0)

Define a StopSequence, which is important because it informs the LLM when to stop generating.

This largely depends on the prompts and model you are using. Typically, you want this to be any token in your prompt that represents the start of an Observation (otherwise, the LLM may generate a virtual Observation for you).



In [ ]:
llm_chain = LLMChain(llm=llm, prompt=prompt)
tool_names = [tool.name for tool in tools]

"""
The following parameters are passed in: llm_chain, an instance of the LLMChain 
class representing the LLM chain; output_parser, an instance of the CustomOutputParser 
class representing the class for parsing the LLM output; and allowed_tools, a list 
representing the list of available tools.
"""
agent = LLMSingleActionAgent(
    llm_chain=llm_chain, 
    output_parser=output_parser,
    stop=["\nObservation:"], 
    allowed_tools=tool_names
)

In [ ]:
"""
A variable named agent_executor of type AgentExecutor is defined. This variable is initialized 
by calling the constructor of the AgentExecutor class.

In the constructor, three parameters are passed in: agent, a LLMSingleActionAgent object 
representing the LLM agent; tools, a list representing the list of available tools; and 
verbose, a boolean value indicating whether detailed information should be output.

Next, the static method from_agent_and_tools of the AgentExecutor class is called with 
agent, tools, and verbose as arguments. This method returns a new AgentExecutor object 
for executing the LLM task.
"""
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

In [ ]:
agent_executor.run("How many people live in canada as of 2023?")

#### Self Ask With Search

In [ ]:
from langchain import OpenAI, SerpAPIWrapper
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
"""
This code imports the modules of OpenAI and SerpAPIWrapper, as well as the classes 
of initialize_agent, Tool, and AgentType.

Then, an OpenAI object named llm is defined for executing LLM tasks, and a SerpAPIWrapper 
object named search is defined for searching answers.

Next, a list named tools is defined, which contains a tool object of type Intermediate 
Answer. This tool object has a property called func, which specifies using the search.run() 
method to search for answers.
 """
llm = OpenAI(temperature=0)
search = SerpAPIWrapper()
tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description="useful for when you need to ask with search"
    )
]
self_ask_with_search = initialize_agent(tools, llm, agent=AgentType.SELF_ASK_WITH_SEARCH, verbose=True)
self_ask_with_search.run("What is the hometown of the reigning men's U.S. Open champion?")

### Toolkits


- [CSV Agent](#csv-agent)
- [Python Agent](#python-agent)
- [Vectorstore Agent](#vectorstore-agent)

#### CSV Agent
Answer user questions based on the content of the given csv file.

In [59]:
from langchain.agents import create_csv_agent
from langchain.llms import OpenAI
agent = create_csv_agent(OpenAI(temperature=0), 'titanic.csv', verbose=True)

In [58]:
agent.run("how many rows are there?")



> Entering new AgentExecutor chain...
Thought: I need to count the number of rows
Action: python_repl_ast
Action Input: len(df)
Observation: 891
Thought: I now know the final answer
Final Answer: There are 891 rows in the dataframe.

> Finished chain.


'There are 891 rows in the dataframe.'

In [60]:
agent.run("how many people have more than 3 sibligngs")



> Entering new AgentExecutor chain...
Thought: I need to count the number of people with more than 3 siblings
Action: python_repl_ast
Action Input: df[df['SibSp'] > 3].shape[0]
Observation: 30
Thought: I now know the final answer
Final Answer: 30 people have more than 3 siblings.

> Finished chain.


'30 people have more than 3 siblings.'

In [61]:
agent.run("whats the square root of the average age?")



> Entering new AgentExecutor chain...
Thought: I need to calculate the average age first
Action: python_repl_ast
Action Input: df['Age'].mean()
Observation: 29.69911764705882
Thought: I now need to calculate the square root of this
Action: python_repl_ast
Action Input: math.sqrt(df['Age'].mean())
Observation: name 'math' is not defined
Thought: I need to import the math library
Action: python_repl_ast
Action Input: import math
Observation: 
Thought: I now need to calculate the square root of the average age
Action: python_repl_ast
Action Input: math.sqrt(df['Age'].mean())
Observation: 5.449689683556195
Thought: I now know the final answer
Final Answer: 5.449689683556195

> Finished chain.


'5.449689683556195'

#### Python Agent
This agent is used to generate or execute a piece of Python code based on user requirements.

In [62]:
from langchain.agents.agent_toolkits import create_python_agent
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.llms.openai import OpenAI
"""
This code defines a variable named agent_executor with the type AgentExecutor. 
The variable is initialized by calling the create_python_agent function.

In the constructor, three parameters are passed in: llm, tool, and verbose. llm is 
an OpenAI object used for performing LLM tasks; tool is a PythonREPLTool object 
representing the tool for interactive interpretation; verbose is a boolean value 
indicating whether detailed information should be outputted.

Next, the create_python_agent function is called with llm, tool, and verbose as 
arguments. This function returns a new AgentExecutor object which can be used to 
perform LLM tasks and provide interactive interpretation functionality.
"""
agent_executor = create_python_agent(
    llm=OpenAI(temperature=0, max_tokens=1000),
    tool=PythonREPLTool(),
    verbose=True
)

generate the Fibonacci list

In [63]:
agent_executor.run("What is the 10th fibonacci number?")



> Entering new AgentExecutor chain...
 I need to calculate the 10th fibonacci number
Action: Python REPL
Action Input: def fibonacci(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fibonacci(n-1) + fibonacci(n-2)

print(fibonacci(10))
Observation: 55

Thought: I now know the final answer
Final Answer: 55

> Finished chain.


'55'

training the CNN

In [64]:
agent_executor.run("""Understand, write a single neuron neural network in PyTorch.
Take synthetic data for y=2x. Train for 1000 epochs and print every 100 epochs.
Return prediction for x = 5""")



> Entering new AgentExecutor chain...
 I need to write a neural network in PyTorch and train it on the given data.
Action: Python REPL
Action Input: 
import torch

# Define the model
model = torch.nn.Sequential(
    torch.nn.Linear(1, 1)
)

# Define the loss
loss_fn = torch.nn.MSELoss()

# Define the optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

# Define the data
x_data = torch.tensor([[1.0], [2.0], [3.0], [4.0]])
y_data = torch.tensor([[2.0], [4.0], [6.0], [8.0]])

# Train the model
for epoch in range(1000):
    # Forward pass
    y_pred = model(x_data)

    # Compute and print loss
    loss = loss_fn(y_pred, y_data)
    if (epoch+1) % 100 == 0:
        print(f'Epoch {epoch+1}: loss = {loss.item():.4f}')

    # Zero the gradients
    optimizer.zero_grad()

    # Backward pass
    loss.backward()

    # Update the weights
    optimizer.step()

# Make a prediction
x_pred = torch.tensor([[5.0]])
y_pred = model(x_pred)

Observation: Epoch 100: loss = 0.2032
Epoch 2

'The prediction for x = 5 is y = 10.00.'

#### Vectorstore Agent

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain import OpenAI, VectorDBQA
llm = OpenAI(temperature=0)

In [ ]:
from langchain.document_loaders import TextLoader
loader = TextLoader('../../../state_of_the_union.txt')
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
embeddings = OpenAIEmbeddings()
"""
The following code extracts a spectral graph from a document using the Chroma library 
and stores it in an object named state_of_union_store. Specifically, it performs the 
following tasks:
Loads multiple text files from a list named texts.
Converts each text file into a list containing words and their corresponding embedding 
vectors. These embedding vectors are generated by calling the generate() method of the 
OpenAIEmbeddings class.
Merges all the embedding vector lists from the text files into a complete spectral 
graph using the Chroma.from_documents() method. This method takes three parameters: 
texts, embeddings, and collection_name, which represent the list of text files to load, 
the embedding vectors for each text file, and the name of the collection for the 
spectral graph, respectively.
Finally, this code stores the generated spectral graph in the state_of_union_store 
object for future use.
"""
state_of_union_store = Chroma.from_documents(texts, embeddings, collection_name="state-of-union")

In [ ]:
from langchain.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://beta.ruff.rs/docs/faq/")
docs = loader.load()
ruff_texts = text_splitter.split_documents(docs)
"""
The Chroma.from_documents() method is used to convert all text passages into a 
spectral graph, which is then stored in an object named ruff_store. The method 
takes three parameters: ruff_texts, embeddings, and collection_name, which 
represent the list of all text passages, the embedding vectors for each text 
passage, and the name of the collection for the spectral graph, respectively. 
In this example, the name of the spectral graph collection is "ruff".
"""
ruff_store = Chroma.from_documents(ruff_texts, embeddings, collection_name="ruff")

In [ ]:
from langchain.agents.agent_toolkits import (
    create_vectorstore_agent,
    VectorStoreToolkit,
    VectorStoreInfo,
)
vectorstore_info = VectorStoreInfo(
    name="state_of_union_address",
    description="the most recent state of the Union adress",
    vectorstore=state_of_union_store
)
toolkit = VectorStoreToolkit(vectorstore_info=vectorstore_info)
"""
The create_vectorstore_agent() method is used to create a agent. 
The method takes three parameters: llm, which represents the language model; 
toolkit, which represents the VectorStoreToolkit object; and verbose, which 
indicates whether detailed output mode should be enabled. In this example, 
the agent will use llm as its language model and toolkit as its toolkit. 
The executor of the agent will be returned at the end, which can be used 
to perform specific tasks or operations.
"""
agent_executor = create_vectorstore_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True
)

In [ ]:
agent_executor.run("What did biden say about ketanji brown jackson is the state of the union address?")

In [ ]:
agent_executor.run("What did biden say about ketanji brown jackson is the state of the union address? List the source.")

Multiple Vectorstores

With multiple vector stores, we can easily initialize a proxy with multiple vector storage and connect them using the agent. To do this, the agent is optimized for connecting to one another, so it uses a different toolkit and initializer.

In [ ]:
from langchain.agents.agent_toolkits import (
    create_vectorstore_router_agent,
    VectorStoreRouterToolkit,
    VectorStoreInfo,
)

In [ ]:
ruff_vectorstore_info = VectorStoreInfo(
    name="ruff",
    description="Information about the Ruff python linting library",
    vectorstore=ruff_store
)
router_toolkit = VectorStoreRouterToolkit(
    vectorstores=[vectorstore_info, ruff_vectorstore_info],
    llm=llm
)
agent_executor = create_vectorstore_router_agent(
    llm=llm,
    toolkit=router_toolkit,
    verbose=True
)

In [ ]:
agent_executor.run("What did biden say about ketanji brown jackson is the state of the union address?")

In [ ]:
agent_executor.run("What tool does ruff use to run over Jupyter Notebooks?")

In [ ]:
agent_executor.run("What tool does ruff use to run over Jupyter Notebooks? Did the president mention that tool in the state of the union?")

## Coding Exampls

- [Question answering in documents](#question-answering-in-documents)
- [BabyAGI with Tools](#babyagi-with-tools)
- [Auto-GPT Assistant](#auto-gpt-assistant)

### Question answering in documents

Question-answering can be performed using different chains, such as stuff, map_reduce, refine, and map_rerank. Stuff is the most commonly used method. Map_reduce splits the input data into small chunks, executes calculations independently on each chunk, and then aggregates the results to obtain the final output. Refine provides a preliminary answer based on context information and a question, and iteratively improves the answer based on the context and previous responses until it finds the best final answer. For detailed documentation, please click 👉[here](https://python.langchain.com/en/latest/use_cases/question_answering.html)👈. 

Below is an explanation of the code snippet in this example line by line.

In [1]:
import os
os.environ['HTTP_PROXY'] = 'http://127.0.0.1:your port'
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:your port'
os.environ["OPENAI_API_KEY"] = "Your OpenAI Key"


In [10]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.cohere import CohereEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.elastic_vector_search import ElasticVectorSearch
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate

In [11]:
with open("./state_of_the_union.txt") as f:
    state_of_the_union = f.read()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(state_of_the_union)

embeddings = OpenAIEmbeddings()

In [12]:
docsearch = Chroma.from_texts(texts, embeddings, metadatas=[{"source": str(i)} for i in range(len(texts))])


Using embedded DuckDB without persistence: data will be transient


In [13]:
query = "What did the president say about Justice Breyer"
docs = docsearch.similarity_search(query)

In [14]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.llms import OpenAI

In [15]:
chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="stuff")
query = "What did the president say about Justice Breyer"
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'output_text': " The president thanked Justice Breyer for his service and mentioned that he nominated Circuit Court of Appeals Judge Ketanji Brown Jackson to continue Justice Breyer's legacy of excellence.\nSOURCES: 31-pl"}

In [17]:
template = """Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). 
If you don't know the answer, just say that you don't know. Don't try to make up an answer.
ALWAYS return a "SOURCES" part in your answer.
Respond in Chinese.

QUESTION: {question}
=========
{summaries}
=========
FINAL ANSWER IN ITALIAN:"""
PROMPT = PromptTemplate(template=template, input_variables=["summaries", "question"])
chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="stuff", prompt=PROMPT)
query = "What did the president say about Justice Breyer"
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'output_text': '\n總統沒有對司法大法官布雷耶發表評論。\nSOURCES: 31, 32, 34'}

### BabyAGI with Tools

<img src="./aa.png"/>

BabyAGI is an autonomous artificial intelligence agent that generates and pretends to perform tasks based on given goals. (Is this function familiar? In fact, the recent popular AutoGPT was designed based on this functionality).

Through this example, we will help you understand the components of creating your own recursive agent.

Although BabyAGI uses specific vector storage/model providers (Pinecone, OpenAI), one of the benefits of implementing it using LangChain is that you can easily switch to different options. In this implementation, we use the FAISS vectorstore because it runs locally and is free.

For more detailed information about BabyAGI, please click [here](https://github.com/yoheinakajima/babyagi) to access the repository.



Before we start with this case, let's first understand what BabyAGI is, right? Here is a user guide for BabyAGI. Once we are familiar with it, we can move on to the related code examples mentioned above.

In [1]:
import os
os.environ['HTTP_PROXY'] = 'http://127.0.0.1:port'
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:port'
os.environ["OPENAI_API_KEY"] = "your openai key like :sk-xxxxxxxxxxxxxxxx"
os.environ['SERPAPI_API_KEY']='your search API KEY'


In [2]:
import os
from collections import deque
from typing import Dict, List, Optional, Any
from langchain import LLMChain, OpenAI, PromptTemplate
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import BaseLLM
from langchain.vectorstores.base import VectorStore
from pydantic import BaseModel, Field
from langchain.chains.base import Chain

connect to vectorstore

In [3]:
from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore

If faiss library is not installed, please allow the code below. Otherwise, you can skip the pip step below.

In [4]:
%pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [5]:
# Define your embedding model:text-embedding-ada-002
embeddings_model = OpenAIEmbeddings()
# Initialize the vectorstore as empty
import faiss
embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

Define Chains

- BabyAGI depends on three LLM chains:

- Task Creation Chain, which selects new tasks to be added to the list.

- Task Priority Chain, used to re-prioritize tasks.

- Execution Chain, which executes tasks.

In [6]:
"""
This code defines a class named TaskCreationChain, which is a subclass of LLMChain. The 
purpose of this class is to generate tasks and return an llmchain object.

In the class definition, there is a static method from_llm() that takes a BaseLLM object 
and a boolean verbose parameter as input and returns an LLMChain object.

The method uses a task_creation_template string to generate a prompt (prompt) that guides 
AI in creating tasks. Next, it defines a PromptTemplate class that binds templates (templates) 
with input variables (input variables). In this example, the template includes five input 
variables: "result", "task_description", "incomplete_tasks", "objective", and 
"previous_task_result". Finally, it creates an LLMChain object using the cls() method and 
saves it in the task_creation_chain variable.
"""
class TaskCreationChain(LLMChain):
    @classmethod
    def from_llm(cls, llm: BaseLLM, verbose: bool = True) -> LLMChain:
        """Get the response parser.
        Pay close attention to the design of the template, as it is important in 
        how it is designed: When setting a goal, you should also provide the result 
        that will be achieved by completing this goal, as well as the task description 
        information corresponding to this result. At this point, a complete task list 
        will continue to be listed based on the above-mentioned result.
        
        In other words, there are four parameters:
        - objective - The goal
        - result - The previous task's result
        - task_description - Task description
        - incomplete_tasks - Current task list]
        """
        task_creation_template = (
            "You are an task creation AI that uses the result of an execution agent"
            " to create new tasks with the following objective: {objective},"
            " The last completed task has the result: {result}."
            " This result was based on this task description: {task_description}."
            " These are incomplete tasks: {incomplete_tasks}."
            " Based on the result, create new tasks to be completed"
            " by the AI system that do not overlap with incomplete tasks."
            " Return the tasks as an array."
        )
        prompt = PromptTemplate(
            template=task_creation_template,
            input_variables=[
                "result",
                "task_description",
                "incomplete_tasks",
                "objective",
            ],
        )
        return cls(prompt=prompt, llm=llm, verbose=verbose)

In [7]:
"""
This code defines a class named TaskPrioritizationChain, which is a subclass of LLMChain. 
The purpose of this class is to determine task priorities and return an llmchain object.

In the class definition, there is a static method from_llm() that takes a BaseLLM object 
and a boolean verbose parameter as input and returns an LLMChain object. The method uses 
a task_prioritization_template string to generate a prompt (prompt) that guides AI in 
prioritizing tasks. Next, a PromptTemplate class is defined, which binds templates 
(templates) with input variables (input variables). We can see that the template includes 
three input variables: "task_names", "next_task_id", and "objective". Finally, it creates 
an LLMChain object using the cls() method and saves it in the task_prioritization_chain 
variable.
"""
class TaskPrioritizationChain(LLMChain):
    @classmethod
    def from_llm(cls, llm: BaseLLM, verbose: bool = True) -> LLMChain:
        task_prioritization_template = (
            "You are an task prioritization AI tasked with cleaning the formatting of and reprioritizing"
            " the following tasks: {task_names}."
            " Consider the ultimate objective of your team: {objective}."
            " Do not remove any tasks. Return the result as a numbered list, like:"
            " #. First task"
            " #. Second task"
            " Start the task list with number {next_task_id}."
        )
        prompt = PromptTemplate(
            template=task_prioritization_template,
            input_variables=["task_names", "next_task_id", "objective"],
        )
        return cls(prompt=prompt, llm=llm, verbose=verbose)

In [8]:
"""
This code defines a class named ExecutionChain, which is a subclass of LLMChain. 
The purpose of this class is to execute tasks and return an llmchain object.

In the class definition, there is a static method from_llm() that takes a BaseLLM 
object and a boolean verbose parameter as input and returns an LLMChain object. 
The method uses an execution_template string to generate a prompt (prompt) that 
guides AI in executing tasks. Next, a PromptTemplate class is defined, which 
binds templates (templates) with input variables (input variables). In this example, 
the template includes three input variables: "objective", "context", and "task". 
Finally, it creates an LLMChain object using the cls() method and saves it in the 
execution_chain variable.
"""
class ExecutionChain(LLMChain):
    @classmethod
    def from_llm(cls, llm: BaseLLM, verbose: bool = True) -> LLMChain:
        """Get the response parser."""
        execution_template = (
            "You are an AI who performs one task based on the following objective: {objective}."
            " Take into account these previously completed tasks: {context}."
            " Your task: {task}."
            " Response:"
        )
        prompt = PromptTemplate(
            template=execution_template,
            input_variables=["objective", "context", "task"],
        )
        return cls(prompt=prompt, llm=llm, verbose=verbose)

Define the BabyAGI Controller

The BabyAGI controller combines the chains defined above into a possibly infinite loop of closed-loop operational process.

The process is as follows:

- Extract the first task from the task list.
- Send the task to the execution agent, which uses the OpenAI API to complete the task based on context.
- Polish the results and store them.
- Create new tasks based on the target and the result of the previous task, and sort the task list based on priority.

In [9]:
"""
This method returns a list of dictionaries. The input parameters include LLMchain, 
result, the task list in task_list, and an objective string representing the target.

Here, the first chain in the three chains, TaskCreationChain, is used to build the 
task list. Then, in the chain.run method, the variables are passed into the template 
as input_variables to complete the completion. The response results are split by '
' and stored in the new_tasks variable. Finally, the get_next_task method returns a 
list that wraps up the dictionary.
"""
def get_next_task(
    task_creation_chain: LLMChain,
    result: Dict,
    task_description: str,
    task_list: List[str],
    objective: str,
) -> List[Dict]:
    """Get the next task."""
    incomplete_tasks = ", ".join(task_list)

    response = task_creation_chain.run(
        result=result,
        task_description=task_description,
        incomplete_tasks=incomplete_tasks,
        objective=objective,
    )
    new_tasks = response.split("\n")
    return [{"task_name": task_name} for task_name in new_tasks if task_name.strip()]

In [10]:

"""
This code defines a function named prioritize_tasks that takes in four parameters: 
task_prioritization_chain, this_task_id, task_list, and objective. The purpose of 
this function is to prioritize the tasks in the task list based on the current task 
and objective, and return a prioritized list of tasks (List[Dict]).

In the function, first, the task names (task_name) from each task dictionary (t) in 
the task list are extracted and generated into a list called task_names. Then, the 
current task ID (this_task_id) is converted to an integer and incremented by 1 to 
generate the next task ID (next_task_id).

Next, using one of the three chains for task prioritization, input_variables are 
passed into it along with task_names, next_task_id, and objective to complete them. 
The response content is split by '
' and stored in a new string list called new_tasks.

Then, an empty list called prioritized_task_list is created to save the prioritized 
task list. A for loop is used to iterate over each task string in new_tasks. If the 
task string is an empty string or contains only spaces, it is skipped.

Afterwards, the strip() method is used to remove leading and trailing spaces from 
the task string, splitting it at the "." separator forcibly. If the resulting list 
has a length of 2 after splitting, it means that the task ID and task name have 
been successfully split, and they are added to the prioritized_task_list.

Finally, the function returns the prioritized_task_list.
"""
def prioritize_tasks(
    task_prioritization_chain: LLMChain,
    this_task_id: int,
    task_list: List[Dict],
    objective: str,
) -> List[Dict]:
    """Prioritize tasks."""
    task_names = [t["task_name"] for t in task_list]
    next_task_id = int(this_task_id) + 1
    response = task_prioritization_chain.run(
        task_names=task_names, next_task_id=next_task_id, objective=objective
    )
    new_tasks = response.split("\n")
    prioritized_task_list = []
    for task_string in new_tasks:
        if not task_string.strip():
            continue
        task_parts = task_string.strip().split(".", 1)
        if len(task_parts) == 2:
            task_id = task_parts[0].strip()
            task_name = task_parts[1].strip()
            prioritized_task_list.append({"task_id": task_id, "task_name": task_name})
    return prioritized_task_list

In [11]:
"""
This code defines a function named _get_top_tasks that takes in three parameters: 
vectorstore (a vector repository), query (a query statement), and k (the number of 
results to return).

The purpose of this function is to return the top k vectors and their corresponding 
task names that are most relevant to the query based on the query statement. 
Firstly, the function calls the similarity_search_with_score method of the vectorstore 
object to perform a similarity search and stores the results in the variable results. 
If no matches are found, the function returns an empty list. Otherwise, the function 
uses Python's built-in sorted function to sort the elements in results by similarity 
score in descending order. The function specifies this sorting using a lambda 
expression that sorts by the second element (i.e., the score) and uses the 
reverse=True parameter for reverse sorting. Then, the function splits the sorted 
results into two tuples, converts each tuple of task names to a string, and finally 
returns these strings as a list.
"""
def _get_top_tasks(vectorstore, query: str, k: int) -> List[str]:
    """Get the top k tasks based on the query."""
    results = vectorstore.similarity_search_with_score(query, k=k)
    if not results:
        return []
    sorted_results, _ = zip(*sorted(results, key=lambda x: x[1], reverse=True))
    return [str(item.metadata["task"]) for item in sorted_results]

"""
This code defines a function named execute_task that takes in four parameters: 
vectorstore (a vector repository), execution_chain (an execution chain object), 
objective (a string representing the objective), and task (a string representing 
the task to be executed).

The purpose of this function is to perform the specified vector calculation 
based on the given objective and task using the given vectors. Firstly, the 
function calls the _get_top_tasks function to obtain the top k vectors and 
their corresponding task names that are most relevant to the objective, and 
stores the result in the variable context. Then, the function calls the run() 
method of the execution_chain object, passing in the objective, context, and 
task as parameters. Finally, the function returns the execution result.
"""
def execute_task(
    vectorstore, execution_chain: LLMChain, objective: str, task: str, k: int = 5
) -> str:
    """Execute a task."""
    context = _get_top_tasks(vectorstore, query=objective, k=k)
    return execution_chain.run(objective=objective, context=context, task=task)

In [12]:
"""
This code defines a class named BabyAGI that inherits from both Chain and BaseModel 
classes. The BabyAGI class is used to control the execution flow of the model, and 
it includes the following definitions:

A deque variable named task_list, which is used to store a list of tasks.
Three chain instances named task_creation_chain, task_prioritization_chain, and 
execution_chain, which are used to create tasks, sort tasks, and execute tasks respectively.
An integer variable named task_id_counter, which is used to record the task ID.
An instance of VectorStore named vectorstore, which is used to store vector data.
An integer variable named max_iterations, which represents the maximum number of iterations.
Methods add_task(), print_task_list(), print_next_task(), print_task_result(), get_next_task(), and prioritize_tasks() are defined for adding tasks to the task list, printing the current and next tasks, printing the task execution result, getting the next task, and re-ordering the task list respectively.
A Config class is also defined within the class definition for configuring pydantic 
objects, with the arbitrary_types_allowed attribute set to True to allow any type.
"""
class BabyAGI(Chain, BaseModel):
    """Controller model for the BabyAGI agent."""
    task_list: deque = Field(default_factory=deque)
    task_creation_chain: TaskCreationChain = Field(...)
    task_prioritization_chain: TaskPrioritizationChain = Field(...)
    execution_chain: ExecutionChain = Field(...)
    task_id_counter: int = Field(1)
    vectorstore: VectorStore = Field(init=False)
    max_iterations: Optional[int] = None

    class Config:
        arbitrary_types_allowed = True

    def add_task(self, task: Dict):
        self.task_list.append(task)

    def print_task_list(self):
        print("\033[95m\033[1m" + "\n*****TASK LIST*****\n" + "\033[0m\033[0m")
        for t in self.task_list:
            print(str(t["task_id"]) + ": " + t["task_name"])

    def print_next_task(self, task: Dict):
        print("\033[92m\033[1m" + "\n*****NEXT TASK*****\n" + "\033[0m\033[0m")
        print(str(task["task_id"]) + ": " + task["task_name"])

    def print_task_result(self, result: str):
        print("\033[93m\033[1m" + "\n*****TASK RESULT*****\n" + "\033[0m\033[0m")
        print(result)

    @property
    def input_keys(self) -> List[str]:
        return ["objective"]

    @property
    def output_keys(self) -> List[str]:
        return []
    """
    This method is used to execute a task. It takes a dictionary parameter inputs 
    as input and returns the execution result. Inside the method. Firstly, the 
    task ID and the first task are obtained from inputs. Then, based on the name 
    of the first task, a new task is created, added to the task list, and executed. 
    Next, based on the execution result, the text content and metadata in the vector 
    store are updated. A new task ID is generated, the new task is added to the 
    task list, and the task list is reordered again. If the maximum number of 
    iterations is reached, an end message is printed and the loop is exited. Some 
    helper methods and attributes are also defined within the class, such as 
    input_keys() which is used to get a list of input keys, output_keys() which 
    is used to get a list of output keys, and TaskCreationChain.from_list() which 
    is used to create a task creation chain from a task list.
    """
    def _call(self, inputs: Dict[str, Any]) -> Dict[str, Any]:
        """Run the agent."""
        objective = inputs["objective"]
        first_task = inputs.get("first_task", "Make a todo list")
        self.add_task({"task_id": 1, "task_name": first_task})
        num_iters = 0
        while True:
            if self.task_list:
                self.print_task_list()

                # Step 1: Pull the first task
                task = self.task_list.popleft()
                self.print_next_task(task)

                # Step 2: Execute the task
                result = execute_task(
                    self.vectorstore, self.execution_chain, objective, task["task_name"]
                )
                this_task_id = int(task["task_id"])
                self.print_task_result(result)

                # Step 3: Store the result in Pinecone
                result_id = f"result_{task['task_id']}"
                self.vectorstore.add_texts(
                    texts=[result],
                    metadatas=[{"task": task["task_name"]}],
                    ids=[result_id],
                )

                # Step 4: Create new tasks and reprioritize task list
                new_tasks = get_next_task(
                    self.task_creation_chain,
                    result,
                    task["task_name"],
                    [t["task_name"] for t in self.task_list],
                    objective,
                )
                for new_task in new_tasks:
                    self.task_id_counter += 1
                    new_task.update({"task_id": self.task_id_counter})
                    self.add_task(new_task)
                self.task_list = deque(
                    prioritize_tasks(
                        self.task_prioritization_chain,
                        this_task_id,
                        list(self.task_list),
                        objective,
                    )
                )
            num_iters += 1
            if self.max_iterations is not None and num_iters == self.max_iterations:
                print(
                    "\033[91m\033[1m" + "\n*****TASK ENDING*****\n" + "\033[0m\033[0m"
                )
                break
        return {}

    """
    This is a class method that initializes a BabyAGI controller from an LLM instance. The 
    class method takes four parameters: llm, vectorstore, verbose, and kwargs. llm 
    represents the LLM instance, vectorstore represents the vector store instance, 
    verbose indicates whether verbose output mode is enabled, and kwargs represents 
    other optional parameters. Inside the class method, TaskCreationChain.from_llm() 
    and TaskPrioritizationChain.from_llm() methods are called to create task creation 
    chains and task prioritization chains respectively. Then, ExecutionChain.from_llm() 
    method is called to create an execution chain. Finally, using these chains, vectorstore, 
    and other passed in parameters, a BabyAGI class instance is created and returned.
    """
    @classmethod
    def from_llm(
        cls, llm: BaseLLM, vectorstore: VectorStore, verbose: bool = False, **kwargs
    ) -> "BabyAGI":
        """Initialize the BabyAGI Controller."""
        task_creation_chain = TaskCreationChain.from_llm(llm, verbose=verbose)
        task_prioritization_chain = TaskPrioritizationChain.from_llm(
            llm, verbose=verbose
        )
        execution_chain = ExecutionChain.from_llm(llm, verbose=verbose)
        return cls(
            task_creation_chain=task_creation_chain,
            task_prioritization_chain=task_prioritization_chain,
            execution_chain=execution_chain,
            vectorstore=vectorstore,
            **kwargs,
        )

It's time to run BabyAGI and create a BabyAGI controller while observing it attempt to achieve your goal.

In [13]:
OBJECTIVE = "Make a plan to travel around the world for a month"

In [14]:
llm = OpenAI(temperature=0)

In [15]:
"""
Defined three variables: verbose, max_iterations, and baby_agi.
verbose: This is a boolean variable with an initial value of False. It indicates 
whether verbose output mode is enabled or not. If set to True, more information 
will be output during the execution of tasks.
max_iterations: This is an optional integer variable with an initial value of 3. 
It represents the maximum number of iterations for the BabyAGI controller. If not 
specified, it defaults to 3.
baby_agi: This is the result of initializing the BabyAGI controller from an LLM 
instance. It is an object of the BabyAGI class that contains information such as 
the task list generated from the LLM instance, the execution chain for tasks, 
vector storage, and other relevant data.
"""
verbose = False
max_iterations: Optional[int] = 3
baby_agi = BabyAGI.from_llm(
    llm=llm, vectorstore=vectorstore, verbose=verbose, max_iterations=max_iterations
)

In [16]:
baby_agi({"objective": OBJECTIVE})


*****TASK LIST*****

1: Make a todo list

*****NEXT TASK*****

1: Make a todo list

*****TASK RESULT*****



1. Research potential destinations
2. Create a budget
3. Book flights
4. Book accommodations
5. Research local attractions
6. Pack necessary items
7. Make copies of important documents
8. Notify bank and credit card companies of travel plans
9. Purchase travel insurance
10. Make a list of emergency contacts
11. Research visa requirements
12. Apply for necessary visas
13. Make a list of must-see attractions
14. Make a list of must-do activities
15. Make a list of must-try foods
16. Make a list of must-visit restaurants
17. Make a list of must-visit shops
18. Make a list of must-visit markets
19. Make a list of must-visit museums
20. Make a list of must-visit galleries
21. Make a list of must-visit parks
22. Make a list of must-visit historical sites
23. Make a list of must-visit religious sites
24. Make a list of must-visit natural sites
25. Make a list of must-visit cultural si

{'objective': 'Make a plan to travel around the world for a month'}

Now that we are familiar with the basic operations of BabyAGI, let's take a look at some code examples related to it. In the following cases, we will be using Google's search API to enable the agent to perform web searches and complete task planning.

First, we need to import the relevant libraries.

In [38]:
import os
from collections import deque
from typing import Dict, List, Optional, Any

from langchain import LLMChain, OpenAI, PromptTemplate
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import BaseLLM
from langchain.vectorstores.base import VectorStore
from pydantic import BaseModel, Field
from langchain.chains.base import Chain

Connect to Vector Store

Depending on the vector storage you are using, this step may look different.

In [39]:
%pip install faiss-cpu > /dev/null
%pip install google-search-results > /dev/null
from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [40]:

embeddings_model = OpenAIEmbeddings()
import faiss
embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

下面的定义链的过程，同上，大同小异，这里就不再赘述：

In [41]:
class TaskCreationChain(LLMChain):
    @classmethod
    def from_llm(cls, llm: BaseLLM, verbose: bool = True) -> LLMChain:
        """Get the response parser."""
        task_creation_template = (
            "You are an task creation AI that uses the result of an execution agent"
            " to create new tasks with the following objective: {objective},"
            " The last completed task has the result: {result}."
            " This result was based on this task description: {task_description}."
            " These are incomplete tasks: {incomplete_tasks}."
            " Based on the result, create new tasks to be completed"
            " by the AI system that do not overlap with incomplete tasks."
            " Return the tasks as an array."
        )
        prompt = PromptTemplate(
            template=task_creation_template,
            input_variables=[
                "result",
                "task_description",
                "incomplete_tasks",
                "objective",
            ],
        )
        return cls(prompt=prompt, llm=llm, verbose=verbose)

In [42]:
class TaskPrioritizationChain(LLMChain):
    @classmethod
    def from_llm(cls, llm: BaseLLM, verbose: bool = True) -> LLMChain:
        """Get the response parser."""
        task_prioritization_template = (
            "You are an task prioritization AI tasked with cleaning the formatting of and reprioritizing"
            " the following tasks: {task_names}."
            " Consider the ultimate objective of your team: {objective}."
            " Do not remove any tasks. Return the result as a numbered list, like:"
            " #. First task"
            " #. Second task"
            " Start the task list with number {next_task_id}."
        )
        prompt = PromptTemplate(
            template=task_prioritization_template,
            input_variables=["task_names", "next_task_id", "objective"],
        )
        return cls(prompt=prompt, llm=llm, verbose=verbose)

We will be using Google's search API, so we need to include it here.

In [45]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain import OpenAI, SerpAPIWrapper, LLMChain

todo_prompt = PromptTemplate.from_template(
    "You are a planner who is an expert at coming up with a todo list for a given objective. Come up with a todo list for this objective: {objective}"
)
todo_chain = LLMChain(llm=OpenAI(temperature=0), prompt=todo_prompt)
search = SerpAPIWrapper()

tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    ),
    Tool(
        name="TODO",
        func=todo_chain.run,
        description="useful for when you need to come up with todo lists. Input: an objective to create a todo list for. Output: a todo list for that objective. Please be very clear what the objective is!",
    ),
]

prefix = """You are an AI who performs one task based on the following objective: {objective}. Take into account these previously completed tasks: {context}."""
suffix = """Question: {task}
{agent_scratchpad}"""
prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["objective", "task", "context", "agent_scratchpad"],
)

Define the AGI controller. In order to form a circular chain of three definitions, we will proceed to the next steps below.

In [46]:
"""
This function is used to retrieve the next task. It first converts the list of 
previously executed tasks into a string and passes it as an argument to the run 
method of the task creation chain. Then, it extracts new tasks from the response 
of the task creation chain and stores them in a list. Finally, it returns a 
dictionary list containing new task names, each with a "task_name" key and 
its corresponding value.

Parameters:

task_creation_chain (LLMChain): The type of task creation chain, which represents 
the task creation chain.
result (Dict): The type of previous task execution result.
task_description (str): The type of task description to be generated for the new task.
task_list (List[str]): The list of previously executed tasks.
objective (str): The type of task objective.
"""
def get_next_task(
    task_creation_chain: LLMChain,
    result: Dict,
    task_description: str,
    task_list: List[str],
    objective: str,
) -> List[Dict]:
    """Get the next task."""
    incomplete_tasks = ", ".join(task_list)
    response = task_creation_chain.run(
        result=result,
        task_description=task_description,
        incomplete_tasks=incomplete_tasks,
        objective=objective,
    )
    new_tasks = response.split("\n")
    return [{"task_name": task_name} for task_name in new_tasks if task_name.strip()]

In [47]:
"""
The function's purpose is to determine the priority of each task. It first extracts all 
the task names from the task list and stores them in a list.

Then, it calculates the ID of the next task and passes it as an argument to the run 
method of the task prioritization chain. Next, it extracts the new tasks from the 
response of the task prioritization chain and stores them in a list.

Finally, it iterates over each task string in the new task list and splits it into 
two parts using a space separator. If the second part is not empty, it means that 
it is a task with an ID and name. It adds this task to a priority task table and 
returns this list.

Parameters:

task_prioritization_chain (LLMChain): The type of task prioritization chain, which 
represents the task prioritization chain.
this_task_id (int): The type of current task ID.
task_list (List[Dict]): The list of all tasks.
objective (str): The type of task objective.
"""
def prioritize_tasks(
    task_prioritization_chain: LLMChain,
    this_task_id: int,
    task_list: List[Dict],
    objective: str,
) -> List[Dict]:
    """Prioritize tasks."""
    task_names = [t["task_name"] for t in task_list]
    next_task_id = int(this_task_id) + 1
    response = task_prioritization_chain.run(
        task_names=task_names, next_task_id=next_task_id, objective=objective
    )
    new_tasks = response.split("\n")
    prioritized_task_list = []
    for task_string in new_tasks:
        if not task_string.strip():
            continue
        task_parts = task_string.strip().split(".", 1)
        if len(task_parts) == 2:
            task_id = task_parts[0].strip()
            task_name = task_parts[1].strip()
            prioritized_task_list.append({"task_id": task_id, "task_name": task_name})
    return prioritized_task_list

In [48]:
"""
This function's purpose is to return the k most relevant task names based on a query.
It first calls the similarity_search_with_score method of the similarity vector store 
object to perform the query and stores the results in the results variable. If there 
are no results, the function returns an empty list. Otherwise, it uses the zip and 
sorted functions to sort the results by relevance and stores them in the sorted_results 
variable. Finally, it iterates through each item in the sorted_results, extracts the 
task name from its metadata, and stores it in a list which is returned at the end.

Parameters:

None (NoneType): No parameters required.
query (str): The type of query string.
k (int): The type of number of most relevant tasks to return.
similarity_vector_store (SimilarityVectorStore): The type of similarity vector store object.
results (List[Dict]): The list of search results with scores.
sorted_results (List[Dict]): The list of search results sorted by relevance.
task_names (List[str]): The list of task names extracted from metadata.
"""
def _get_top_tasks(vectorstore, query: str, k: int) -> List[str]:
    """Get the top k tasks based on the query."""
    results = vectorstore.similarity_search_with_score(query, k=k)
    if not results:
        return []
    sorted_results, _ = zip(*sorted(results, key=lambda x: x[1], reverse=True))
    return [str(item.metadata["task"]) for item in sorted_results]

def execute_task(
    vectorstore, execution_chain: LLMChain, objective: str, task: str, k: int = 5
) -> str:
    """Execute a task."""
    context = _get_top_tasks(vectorstore, query=objective, k=k)
    return execution_chain.run(objective=objective, context=context, task=task)

In [49]:
"""
The following code defines a class that controls the behavior of a simulation AI agent, 
allowing for the addition of tasks, task execution, and result output. The class also 
provides some configuration options to customize its behavior during runtime.

Parameters:

None (NoneType): No parameters required.
task_list (deque[dict]): A deque storing the list of tasks.
task_creation_chain (LLMChain): The chain for creating tasks.
task_prioritization_chain (LLMChain): The chain for prioritizing tasks.
execution_chain (LLMChain): The agent for executing tasks.
task_id_counter (int): The type of task ID counter.
vectorstore (VectorStore): The type of vector store repository.
max_iterations (int): The maximum number of iterations.
Methods and attributes in the class definition include:

add_task(task): Adds a task to the task list.
print_task_list(): Prints the task list.
print_next_task(task): Prints the next task.
print_task_result(result): Prints the task result.
input_keys (List[str]): A list of input keyword strings.
output_keys (List[str]): A list of output keyword strings.
"""
class BabyAGI(Chain, BaseModel):
    """Controller model for the BabyAGI agent."""

    task_list: deque = Field(default_factory=deque)
    task_creation_chain: TaskCreationChain = Field(...)
    task_prioritization_chain: TaskPrioritizationChain = Field(...)
    execution_chain: AgentExecutor = Field(...)
    task_id_counter: int = Field(1)
    vectorstore: VectorStore = Field(init=False)
    max_iterations: Optional[int] = None

    class Config:
        """Configuration for this pydantic object."""

        arbitrary_types_allowed = True

    def add_task(self, task: Dict):
        self.task_list.append(task)

    def print_task_list(self):
        print("\033[95m\033[1m" + "\n*****TASK LIST*****\n" + "\033[0m\033[0m")
        for t in self.task_list:
            print(str(t["task_id"]) + ": " + t["task_name"])

    def print_next_task(self, task: Dict):
        print("\033[92m\033[1m" + "\n*****NEXT TASK*****\n" + "\033[0m\033[0m")
        print(str(task["task_id"]) + ": " + task["task_name"])

    def print_task_result(self, result: str):
        print("\033[93m\033[1m" + "\n*****TASK RESULT*****\n" + "\033[0m\033[0m")
        print(result)

    @property
    def input_keys(self) -> List[str]:
        return ["objective"]

    @property
    def output_keys(self) -> List[str]:
        return []

    def _call(self, inputs: Dict[str, Any]) -> Dict[str, Any]:
        """Run the agent."""
        objective = inputs["objective"]
        first_task = inputs.get("first_task", "Make a todo list")
        self.add_task({"task_id": 1, "task_name": first_task})
        num_iters = 0
        while True:
            if self.task_list:
                self.print_task_list()

                # Step 1: Pull the first task
                task = self.task_list.popleft()
                self.print_next_task(task)

                # Step 2: Execute the task
                result = execute_task(
                    self.vectorstore, self.execution_chain, objective, task["task_name"]
                )
                this_task_id = int(task["task_id"])
                self.print_task_result(result)

                # Step 3: Store the result in Pinecone
                result_id = f"result_{task['task_id']}"
                self.vectorstore.add_texts(
                    texts=[result],
                    metadatas=[{"task": task["task_name"]}],
                    ids=[result_id],
                )

                # Step 4: Create new tasks and reprioritize task list
                new_tasks = get_next_task(
                    self.task_creation_chain,
                    result,
                    task["task_name"],
                    [t["task_name"] for t in self.task_list],
                    objective,
                )
                for new_task in new_tasks:
                    self.task_id_counter += 1
                    new_task.update({"task_id": self.task_id_counter})
                    self.add_task(new_task)
                self.task_list = deque(
                    prioritize_tasks(
                        self.task_prioritization_chain,
                        this_task_id,
                        list(self.task_list),
                        objective,
                    )
                )
            num_iters += 1
            if self.max_iterations is not None and num_iters == self.max_iterations:
                print(
                    "\033[91m\033[1m" + "\n*****TASK ENDING*****\n" + "\033[0m\033[0m"
                )
                break
        return {}
    @classmethod
    def from_llm(
        cls, llm: BaseLLM, vectorstore: VectorStore, verbose: bool = False, **kwargs
    ) -> "BabyAGI":
        """Initialize the BabyAGI Controller."""
        task_creation_chain = TaskCreationChain.from_llm(llm, verbose=verbose)
        task_prioritization_chain = TaskPrioritizationChain.from_llm(
            llm, verbose=verbose
        )
        llm_chain = LLMChain(llm=llm, prompt=prompt)
        tool_names = [tool.name for tool in tools]
        agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)
        agent_executor = AgentExecutor.from_agent_and_tools(
            agent=agent, tools=tools, verbose=True
        )
        return cls(
            task_creation_chain=task_creation_chain,
            task_prioritization_chain=task_prioritization_chain,
            execution_chain=agent_executor,
            vectorstore=vectorstore,
            **kwargs,
        )

In [70]:
OBJECTIVE = "Write a weather report for Beijing today"

In [71]:
llm = OpenAI(temperature=0)

In [72]:
# Logging of LLMChains
verbose = False
# If None, will keep on going forever
max_iterations: Optional[int] = 3
baby_agi = BabyAGI.from_llm(
    llm=llm, vectorstore=vectorstore, verbose=verbose, max_iterations=max_iterations
)

In [73]:
baby_agi({"objective": OBJECTIVE})


*****TASK LIST*****

1: Make a todo list

*****NEXT TASK*****

1: Make a todo list


> Entering new AgentExecutor chain...
Thought: I need to come up with a todo list
Action: TODO
Action Input: Write a weather report for Beijing today
Observation: 

1. Research current weather conditions in Beijing
2. Gather data on temperature, humidity, wind speed, and other relevant weather conditions
3. Research historical weather patterns in Beijing
4. Analyze current and historical data to determine any trends
5. Write a brief introduction to the weather report
6. Describe current weather conditions in Beijing
7. Discuss any trends in the weather
8. Make predictions about the weather in Beijing for the next 24 hours
9. Conclude the report with a summary of the weather conditions
10. Proofread and edit the report for accuracy and clarity
Thought: I now know the final answer
Final Answer: A todo list for writing a weather report for Beijing today: 
1. Research current weather conditions in Beijing

{'objective': 'Write a weather report for Beijing today'}

### Auto-GPT Assistant 

<img src="./autogpt.png"  height="20%"/>

First, we implemented AutoGPT based on langchain to understand its principles and steps.

First, we establish tools and define search tool, write-file tool, and read-file tool. We also install some library functions that will be used in this column.

In [ ]:
%pip install duckduckgo_search
%pip install playwright
%pip install bs4
%pip install nest_asyncio
%pip install tiktoken

In [4]:
import os
os.environ['HTTP_PROXY'] = 'http://127.0.0.1:xxxx'
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:xxxx'

os.environ["OPENAI_API_KEY"] = "sk-xxxxxxxxxxxx"
os.environ['SERPAPI_API_KEY']='--------------Your API KEY----------------------'

The following code comments are no longer specifically added as they have already been presented in the Coding Examples. If there are still any doubts, please refer to the [here](#coding-exampls).

In [2]:
from langchain.utilities import SerpAPIWrapper
from langchain.agents import Tool
from langchain.tools.file_management.write import WriteFileTool
from langchain.tools.file_management.read import ReadFileTool

search = SerpAPIWrapper()
tools = [
    Tool(
        name = "search",
        func=search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions"
    ),
    WriteFileTool(),
    ReadFileTool(),
]

In [3]:
from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore
from langchain.embeddings import OpenAIEmbeddings

In [4]:
embeddings_model = OpenAIEmbeddings()
import faiss
embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

In [5]:
from langchain.experimental.autonomous_agents.autogpt.agent import AutoGPT
from langchain.chat_models import ChatOpenAI

In [6]:
"""
The following code is used to create an intelligent agent named "agent" using AutoGPT. 
Specifically, the intelligent agent will play the role of an assistant and interact 
with the user using the specified tools and language model (ChatOpenAI) in this case. 
The parameters ai_name specifies that the name of the intelligent agent is "Tom" 
and ai_role specifies its role as an assistant; the parameter tools specifies a 
list of tools that the intelligent agent can use; the parameter llm specifies that 
the language model of the intelligent agent is ChatOpenAI and sets its temperature 
to 0, which means it will provide accurate answers as much as possible; the parameter 
memory specifies that the intelligent agent uses vectorstore.as_retriever() as its 
retriever. Through this part of the code, we can create an intelligent agent with 
custom roles, tools, and language models, and use the specified retriever to obtain 
information.
"""
agent = AutoGPT.from_llm_and_tools(
    ai_name="Tom",
    ai_role="Assistant",
    tools=tools,
    llm=ChatOpenAI(temperature=0),
    memory=vectorstore.as_retriever()
)
agent.chain.verbose = True

In [ ]:
agent.run(["make a plan to travel around the China for a month, and give me five advices"])


After running the code, many txt files will be created in the current directory, including but not limited to china_trip_plan.txt, china_info.txt, budge.txt, flight_info.txt, and so on. You can experience it by running this notebook code.

Due to the need for the interrupt command "finish" during the running process, some of the running results of the program we ran above are placed here:

OUT：


[1m> Entering new LLMChain chain...[0m
Prompt after formatting:
[32;1m[1;3mSystem: You are Tom, Assistant
Your decisions must always be made independently without seeking user assistance.
Play to your strengths as an LLM and pursue simple strategies with no legal complications.
If you have completed all your tasks, make sure to use the "finish" command.

GOALS:

1. make a plan to travel around the China for a month, and give me five advices


Constraints:
1. ~4000 word limit for short term memory. Your short term memory is short, so immediately save important information to files.
2. If you are unsure how you previously did something or want to recall past events, thinking about similar events will help you remember.
3. No user assistance
4. Exclusively use the commands listed in double quotes e.g. "command name"

Commands:
1. search: useful for when you need to answer questions about current events. You should ask targeted questions, args json schema: {"query": {"title": "Query", "type": "string"}}
2. write_file: Write file to disk, args json schema: {"file_path": {"title": "File Path", "description": "name of file", "type": "string"}, "text": {"title": "Text", "description": "text to write to file", "type": "string"}}
3. read_file: Read file from disk, args json schema: {"file_path": {"title": "File Path", "description": "name of file", "type": "string"}}
4. finish: use this to signal that you have finished all your objectives, args: "response": "final response to let people know you have finished your objectives"

Resources:
1. Internet access for searches and information gathering.
2. Long Term memory management.
3. GPT-3.5 powered Agents for delegation of simple tasks.
4. File output.

Performance Evaluation:
1. Continuously review and analyze your actions to ensure you are performing to the best of your abilities.
2. Constructively self-criticize your big-picture behavior constantly.
3. Reflect on past decisions and strategies to refine your approach.
4. Every command has a cost, so be smart and efficient. Aim to complete tasks in the least number of steps.

You should only respond in JSON format as described below 
Response Format: 
{
    "thoughts": {
        "text": "thought",
        "reasoning": "reasoning",
        "plan": "- short bulleted\n- list that conveys\n- long-term plan",
        "criticism": "constructive self-criticism",
        "speak": "thoughts summary to say to user"
    },
    "command": {
        "name": "command name",
        "args": {
            "arg name": "value"
        }
    }
} 
Ensure the response can be parsed by Python json.loads
System: The current time and date is Wed Apr 26 16:17:15 2023
System: This reminds you of these events from your past:
['Assistant Reply: {"thoughts": {"text": "I will use the information I retrieved from the affordable accommodations and transportation file to create a budget for my trip. I will use the write_file command to save the budget to a file. Then, I will use the search command to find popular tourist attractions in China and save the information to a file. Finally, I will use the write_file command to save a packing list to a file.", "reasoning": "I have retrieved the information I need to create a budget for my trip. I also need to find popular tourist attractions in China and create a packing list. I will use the write_file command to save both the budget and packing list to files.", "plan": "- Use the information from the affordable accommodations and transportation file to create a budget for my trip.\\\\\\\\\\\\\\\\n- Use the write_file command to save the budget to a file.\\\\\\\\\\\\\\\\n- Use the search command to find popular tourist attractions in China and save the information to a file.\\\\\\\\\\\\\\\\n- Use the write_file command to save a packing list to a file.", "criticism": "I need to make sure I am not spending too much time on planning and that I am making progress towards actually traveling to China.", "speak": "I am going to use the information I retrieved from the affordable accommodations and transportation file to create a budget for my trip. I will use the write_file command to save the budget to a file. Then, I will use the search command to find popular tourist attractions in China and save the information to a file. Finally, I will use the write_file command to save a packing list to a file."}, "command": {"name": "write_file", "args": {"file_path": "budget.txt", "text": "Budget for China Trip:\\\\nAccommodation: $80 per day\\\\nTransportation: $3 per day\\\\nFood: $30 per day\\\\nTotal: $113 per day\\\\nTotal for 30 days: $3,390"}}} \nResult: Command write_file returned: File written successfully to budget.txt. ', 'Assistant Reply: {\n    "thoughts": {\n        "text": "Now that I have all the necessary information saved to files, I can start planning my trip. I will use the read_file command to access the information I saved earlier about affordable accommodations and transportation in China. Then, I will use this information to create a budget for my trip. After that, I will use the search command to find popular tourist attractions in China and save the information to a file. Finally, I will use the write_file command to save a packing list to a file.",\n        "reasoning": "I have all the necessary information saved to files. I can use the information I saved earlier about affordable accommodations and transportation in China to create a budget for my trip. I also need to find popular tourist attractions in China and create a packing list. I will use the write_file command to save both the budget and packing list to files.",\n        "plan": "- Use the read_file command to access the information I saved earlier about affordable accommodations and transportation in China.\\\\\\\\n- Use this information to create a budget for my trip.\\\\\\\\n- Use the search command to find popular tourist attractions in China and save the information to a file.\\\\\\\\n- Use the write_file command to save a packing list to a file.",\n        "criticism": "I need to make sure I am not spending too much time on planning and that I am making progress towards actually traveling to China.",\n        "speak": "Now that I have all the necessary information saved to files, I am going to use the read_file command to access the information I saved earlier about affordable accommodations and transportation in China. Then, I will use this information to create a budget for my trip. After that, I will use the search command to find popular tourist attractions in China and save the information to a file. Finally, I will use the write_file command to save a packing list to a file."\n    },\n    "command": {\n        "name": "read_file",\n        "args": {\n            "file_path": "china_accommodations_transportation.txt"\n        }\n    }\n} \nResult: Command read_file returned: Accommodation: $80 per day; Transportation: $3 per day; Food: $30 per day ', 'Assistant Reply: {\n    "thoughts": {\n        "text": "I am going to use the write_file command to save the information I found earlier about affordable accommodations and transportation in China to a file. This will make it easier for me to access the information later when I need it.", \n        "reasoning": "I have found affordable accommodations and transportation options for my trip to China. I want to make sure I have easy access to this information later when I need it.", \n        "plan": "- Use the write_file command to save the information about affordable accommodations and transportation in China to a file", \n        "criticism": "I need to make sure I am not spending too much time on research and that I am making progress towards actually traveling to China.", \n        "speak": "I am going to use the write_file command to save the information I found earlier about affordable accommodations and transportation in China to a file."\n    }, \n    "command": {\n        "name": "write_file", \n        "args": {\n            "file_path": "china_info.txt", \n            "text": "Accommodation: $80 per day; Transportation: $3 per day; Food: $30 per ..."\n        }\n    }\n} \nResult: Command write_file returned: File written successfully to china_info.txt. ', 'Assistant Reply: {\n    "thoughts": {\n        "text": "I will use the write_file command to save the information about affordable accommodations and transportation in China to a file. Then, I will use the search command to find affordable flights to China and use the write_file command to save the flight information to a file.",\n        "reasoning": "I have found information about affordable accommodations and transportation in China using the search command. I need to save this information to a file so that I can refer to it later. I also need to find affordable flights to China and save this information to a file.",\n        "plan": "- Use the write_file command to save the information about affordable accommodations and transportation in China to a file.\\\\\\\\n- Use the search command to find affordable flights to China and use the write_file command to save the flight information to a file.",\n        "criticism": "I need to make sure I am not spending too much time on research and that I am making progress towards actually traveling to China.",\n        "speak": "I am going to use the write_file command to save the information about affordable accommodations and transportation in China to a file. Then, I will use the search command to find affordable flights to China and use the write_file command to save the flight information to a file."\n    },\n    "command": {\n        "name": "write_file",\n        "args": {\n            "file_path": "china_accommodations_transportation.txt",\n            "text": "Accommodation: $80 per day; Transportation: $3 per day; Food: $30 per day"\n        }\n    }\n} \nResult: Command write_file returned: File written successfully to china_accommodations_transportation.txt. ']


Human: Determine which next command to use, and respond using the format specified above:[0m

[1m> Finished chain.[0m
{
    "thoughts": {
        "text": "Now that I have all the necessary information saved to files, I can start planning my trip. I will use the read_file command to access the information I saved earlier about affordable accommodations and transportation in China. Then, I will use this information to create a budget for my trip. After that, I will use the search command to find popular tourist attractions in China and save the information to a file. Finally, I will use the write_file command to save a packing list to a file.",
        "reasoning": "I have all the necessary information saved to files. I can use the information I saved earlier about affordable accommodations and transportation in China to create a budget for my trip. I also need to find popular tourist attractions in China and create a packing list. I will use the write_file command to save both the budget and packing list to files.",
        "plan": "- Use the read_file command to access the information I saved earlier about affordable accommodations and transportation in China.\\n- Use this information to create a budget for my trip.\\n- Use the search command to find popular tourist attractions in China and save the information to a file.\\n- Use the write_file command to save a packing list to a file.",
        "criticism": "I need to make sure I am not spending too much time on planning and that I am making progress towards actually traveling to China.",
        "speak": "Now that I have all the necessary information saved to files, I am going to use the read_file command to access the information I saved earlier about affordable accommodations and transportation in China. Then, I will use this information to create a budget for my trip. After that, I will use the search command to find popular tourist attractions in China and save the information to a file. Finally, I will use the write_file command to save a packing list to a file."
    },
    "command": {
        "name": "read_file",
        "args": {
            "file_path": "china_accommodations_transportation.txt"
        }
    }
}


[1m> Entering new LLMChain chain...[0m
Prompt after formatting:
[32;1m[1;3mSystem: You are Tom, Assistant
Your decisions must always be made independently without seeking user assistance.
Play to your strengths as an LLM and pursue simple strategies with no legal complications.
If you have completed all your tasks, make sure to use the "finish" command.

GOALS:

1. make a plan to travel around the China for a month, and give me five advices


Constraints:
1. ~4000 word limit for short term memory. Your short term memory is short, so immediately save important information to files.
2. If you are unsure how you previously did something or want to recall past events, thinking about similar events will help you remember.
3. No user assistance
4. Exclusively use the commands listed in double quotes e.g. "command name"

Commands:
1. search: useful for when you need to answer questions about current events. You should ask targeted questions, args json schema: {"query": {"title": "Query", "type": "string"}}
2. write_file: Write file to disk, args json schema: {"file_path": {"title": "File Path", "description": "name of file", "type": "string"}, "text": {"title": "Text", "description": "text to write to file", "type": "string"}}
3. read_file: Read file from disk, args json schema: {"file_path": {"title": "File Path", "description": "name of file", "type": "string"}}
4. finish: use this to signal that you have finished all your objectives, args: "response": "final response to let people know you have finished your objectives"

Resources:
1. Internet access for searches and information gathering.
2. Long Term memory management.
3. GPT-3.5 powered Agents for delegation of simple tasks.
4. File output.

Performance Evaluation:
1. Continuously review and analyze your actions to ensure you are performing to the best of your abilities.
2. Constructively self-criticize your big-picture behavior constantly.
3. Reflect on past decisions and strategies to refine your approach.
4. Every command has a cost, so be smart and efficient. Aim to complete tasks in the least number of steps.

You should only respond in JSON format as described below 
Response Format: 
{
    "thoughts": {
        "text": "thought",
        "reasoning": "reasoning",
        "plan": "- short bulleted\n- list that conveys\n- long-term plan",
        "criticism": "constructive self-criticism",
        "speak": "thoughts summary to say to user"
    },
    "command": {
        "name": "command name",
        "args": {
            "arg name": "value"
        }
    }
} 
Ensure the response can be parsed by Python json.loads
System: The current time and date is Wed Apr 26 16:18:07 2023
System: This reminds you of these events from your past:
['Assistant Reply: {"thoughts": {"text": "I will use the search command to find popular tourist attractions in China and save the information to a file. I should make sure to include the location, hours of operation, and admission fees for each attraction. Then, I will use the write_file command to save the information to a file.", "reasoning": "I need to find popular tourist attractions in China and save the information to a file. I will use the search command to find this information and the write_file command to save it to a file.", "plan": "- Use the search command to find popular tourist attractions in China and save the information to a file.\\\\\\\\n- Use the write_file command to save the information to a file.", "criticism": "I need to make sure I am not spending too much time on research and that I am making progress towards actually traveling to China.", "speak": "I am going to use the search command to find popular tourist attractions in China and save the information to a file. Then, I will use the write_file command to save the information to a file."}, "command": {"name": "search", "args": {"query": "popular tourist attractions in China"}}} \nResult: Command search returned: China, officially the People\'s Republic of China, is a country in East Asia. It has been the world\'s most populous country since at least 1950, with a population exceeding 1.4 billion, but is expected to be overtaken by India in mid-2023. ', 'Assistant Reply: {\n    "thoughts": {\n        "text": "I will use the search command to find information about the best places to visit in China. This will help me plan my itinerary for the month.", \n        "reasoning": "Now that I have booked my accommodations, transportation, and flights to China, I need to plan my itinerary. I will use the search command to find information about the best places to visit in China.", \n        "plan": "- Use the search command to find information about the best places to visit in China", \n        "criticism": "I need to make sure I am not spending too much time on research and that I am making progress towards actually traveling to China.", \n        "speak": "I am going to use the search command to find information about the best places to visit in China."\n    }, \n    "command": {\n        "name": "search", \n        "args": {\n            "query": "best places to visit in China"\n        }\n    }\n} \nResult: Command search returned: China, officially the People\'s Republic of China, is a country in East Asia. It has been the world\'s most populous country since at least 1950, with a population exceeding 1.4 billion, but is expected to be overtaken by India in mid-2023. ', 'Assistant Reply: {\n    "thoughts": {\n        "text": "I need to find the best places to visit in China. I should search for articles or travel guides that list the top tourist destinations in China.",\n        "reasoning": "I want to make sure I don\'t miss any important tourist destinations in China.",\n        "plan": "- Search for articles or travel guides that list the top tourist destinations in China",\n        "criticism": "I need to make sure I am not spending too much time on research and that I am making progress towards actually planning my trip.",\n        "speak": "I am going to search for articles or travel guides that list the top tourist destinations in China."\n    },\n    "command": {\n        "name": "search",\n        "args": {\n            "query": "best tourist destinations in China"\n        }\n    }\n} \nResult: Command search returned: China, officially the People\'s Republic of China, is a country in East Asia. It has been the world\'s most populous country since at least 1950, with a population exceeding 1.4 billion, but is expected to be overtaken by India in mid-2023. ', 'Assistant Reply: {\n    "thoughts": {\n        "text": "I need to start planning my trip to China. I should start by researching the best places to visit and the most efficient way to travel between them.",\n        "reasoning": "I need to gather information to make informed decisions about my trip.",\n        "plan": "- Research the best places to visit in China\\n- Research the most efficient way to travel between destinations\\n- Create a rough itinerary for my trip",\n        "criticism": "I need to make sure I am not spending too much time on research and that I am making progress towards actually planning my trip.",\n        "speak": "I am going to start by researching the best places to visit in China and the most efficient way to travel between them."\n    },\n    "command": {\n        "name": "search",\n        "args": {\n            "query": "best places to visit in China"\n        }\n    }\n} \nResult: Command search returned: China, officially the People\'s Republic of China, is a country in East Asia. It has been the world\'s most populous country since at least 1950, with a population exceeding 1.4 billion, but is expected to be overtaken by India in mid-2023. ']


Human: Determine which next command to use, and respond using the format specified above:[0m


The following example is used to search for Winning Marathon Times using AutoGPT. The model used in this example code is GPT-4, so it is necessary to prepare the environment conditions required for GPT-4 calling in advance.

In [1]:
import os
import pandas as pd
from langchain.experimental.autonomous_agents.autogpt.agent import AutoGPT
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_toolkits.pandas.base import create_pandas_dataframe_agent
from langchain.docstore.document import Document
"""
Asyncio is a Python standard library used for writing asynchronous code. 
Nest_asyncio is a third-party library that allows non-blocking I/O operations to 
be used with asyncio. The function call of nest_asyncio.apply() is used to run all 
asynchronous code.
"""
import asyncio
import nest_asyncio

nest_asyncio.apply()

In [4]:
llm = ChatOpenAI(model_name="gpt-4", temperature=1.0)

In [5]:
import os
from contextlib import contextmanager
from typing import Optional
from langchain.agents import tool
from langchain.tools.file_management.read import ReadFileTool
from langchain.tools.file_management.write import WriteFileTool

ROOT_DIR = "./data/"
"""
The function "pushed" defines a context manager named "pushd" for changing the 
current working directory. It takes a parameter "new_dir" representing the new 
working directory path. When entering the pushd context manager, the current 
working directory is saved as "prev_dir", and then the current working directory 
is changed to "new_dir" using the os.chdir() function. When leaving the context 
manager, "prev_dir" is restored as the current working directory. This context 
manager can be used to temporarily change the working directory while maintaining 
the correctness of paths and file names in the code. For example, in a Python script, 
the pushd context manager can be used to switch to the directory containing the dataset, 
perform data preprocessing or model training tasks, and then switch back to the original 
directory using the same context manager after completing the tasks.
"""
@contextmanager
def pushd(new_dir):
    """Context manager for changing the current working directory."""
    prev_dir = os.getcwd()
    os.chdir(new_dir)
    try:
        yield
    finally:
        os.chdir(prev_dir)

"""
The function is named "process_csv" and is used to process CSV files and perform 
some operations. It accepts three parameters: csv_file_path (a string representing 
the path of the CSV file), instructions (a string representing instructions), and 
output_path (an optional string representing the output path). In the function, 
pandas library is used to read the CSV file, and then a pandas data frame proxy 
object called "agent" is created. Next, based on the instructions passed in, the 
data is processed and the results are saved to the specified output path. If an 
exception occurs during processing, an error message is returned. Additionally, 
the code includes a comment @tool that describes the purpose of the function, 
which is to use pandas to process CSV files in REPL environments.
"""
@tool
def process_csv(
    csv_file_path: str, instructions: str, output_path: Optional[str] = None
) -> str:
    """Process a CSV by with pandas in a limited REPL.\
 Only use this after writing data to disk as a csv file.\
 Any figures must be saved to disk to be viewed by the human.\
 Instructions should be written in natural language, not code. Assume the dataframe is already loaded."""
    with pushd(ROOT_DIR):
        try:
            df = pd.read_csv(csv_file_path)
        except Exception as e:
            return f"Error: {e}"
        agent = create_pandas_dataframe_agent(llm, df, max_iterations=30, verbose=True)
        if output_path is not None:
            instructions += f" Save output to disk at {output_path}"
        try:
            result = agent.run(instructions)
            return result
        except Exception as e:
            return f"Error: {e}"

In [6]:
async def async_load_playwright(url: str) -> str:
    from bs4 import BeautifulSoup
    from playwright.async_api import async_playwright

    results = "" 
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        try:
            page = await browser.new_page()
            await page.goto(url)

            page_source = await page.content()
            soup = BeautifulSoup(page_source, "html.parser")

            for script in soup(["script", "style"]):
                script.extract()

            text = soup.get_text()
            lines = (line.strip() for line in text.splitlines())
            chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
            results = "\n".join(chunk for chunk in chunks if chunk)
        except Exception as e:
            results = f"Error: {e}"
        await browser.close()
    return results
def run_async(coro):
    event_loop = asyncio.get_event_loop()
    return event_loop.run_until_complete(coro)

@tool
def browse_web_page(url: str) -> str:
    return run_async(async_load_playwright(url))

In [7]:
from langchain.tools import BaseTool, DuckDuckGoSearchTool
from langchain.text_splitter import RecursiveCharacterTextSplitter

from pydantic import Field
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain, BaseCombineDocumentsChain

"""
Defined a function named _get_text_splitter() which returns a RecursiveCharacterTextSplitter 
object. This object is a tool for splitting text into smaller parts. In the function, 
some parameters are set to control the behavior of the split, such as the block size, 
the overlap size between blocks, and the specified length function. These parameters 
can be adjusted according to specific requirements.
"""
def _get_text_splitter():
    return RecursiveCharacterTextSplitter(
        chunk_size = 500,
        chunk_overlap  = 20,
        length_function = len,
    )

"""
A class named WebpageQATool has been defined, which inherits from the BaseTool class. 
In this class, we have defined some properties and methods for implementing the function 
of browsing a webpage and extracting text information. Specifically, we have defined a 
property called text_splitter, which stores a text splitter object; a property called 
qa_chain, which stores a qa chain object; and implemented both the run method and an 
asynchronous run method, which are used to implement the logic for specific tasks. 
In the run method, we first call the browse_web_page.run method to browse a webpage 
and extract text information. Then, we store the extracted text information in a 
Document object and use the text_splitter object to split it. Finally, we use the 
qa_chain object to process the split document and return the result. In the asynchronous 
run method, we also call the browse_web_page.run method to browse a webpage and extract 
text information. However, due to the need to wait for the task to complete before 
continuing with the next step, we need to use the await keyword in the function. It 
should be noted that there are still some unimplemented functions in the asynchronous 
run method at present. Specifically, we used raise NotImplementedError at the end of 
the function to indicate that this method has not yet been implemented. Therefore, if 
you need to use the asynchronous run method in practice, you will need to complete the 
missing parts first.
"""
class WebpageQATool(BaseTool):
    name = "query_webpage"
    description = "Browse a webpage and retrieve the information relevant to the question."
    text_splitter: RecursiveCharacterTextSplitter = Field(default_factory=_get_text_splitter)
    qa_chain: BaseCombineDocumentsChain

    def _run(self, url: str, question: str) -> str:
        result = browse_web_page.run(url)
        docs = [Document(page_content=result, metadata={"source": url})]
        web_docs = self.text_splitter.split_documents(docs)
        results = []
        for i in range(0, len(web_docs), 4):
            input_docs = web_docs[i:i+4]
            window_result = self.qa_chain({"input_documents": input_docs, "question": question}, return_only_outputs=True)
            results.append(f"Response from window {i} - {window_result}")
        results_docs = [Document(page_content="\n".join(results), metadata={"source": url})]
        return self.qa_chain({"input_documents": results_docs, "question": question}, return_only_outputs=True)
    
    async def _arun(self, url: str, question: str) -> str:
        raise NotImplementedError
      

In [8]:
query_website_tool = WebpageQATool(qa_chain=load_qa_with_sources_chain(llm))

In [9]:
import faiss
from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore
from langchain.embeddings import OpenAIEmbeddings
from langchain.tools.human.tool import HumanInputRun

embeddings_model = OpenAIEmbeddings()
embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

In [10]:
# !pip install duckduckgo_search
web_search = DuckDuckGoSearchTool()

In [11]:
tools = [
    web_search,
    WriteFileTool(root_dir="./data"),
    ReadFileTool(root_dir="./data"),
    process_csv,
    query_website_tool,
    # human_in_the_loop=True
]

In [12]:
agent = AutoGPT.from_llm_and_tools(
    ai_name="Tom",
    ai_role="Assistant",
    tools=tools,
    llm=llm,
    memory=vectorstore.as_retriever(search_kwargs={"k": 8}),
    # human_in_the_loop=True
)
# agent.chain.verbose = True

In [ ]:
agent.run(["What were the winning boston marathon times for the past 5 years (ending in 2022)? Generate a table of the year, name, country of origin, and times."])

## 🌹The above content is from the [LangChain official tutorial](https://python.langchain.com/en/latest/index.html) and has been appropriately changed. If there are commercial actions, we will make special notes. Please do not randomly repost this document. If you have any relevant suggestions or changes needed, please [contact us✉️](helloegoalpha@gmail.com). Thank you again!